## **REPORTE KPIS COMERCIAL**

#### **Librerías**

In [ ]:
import os
import math
import time
import smtplib

import numpy as np
import pandas as pd

import xlsxwriter

from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from google.cloud import bigquery

#### **Funciones**

ModuleNotFoundError: No module named 'xlsxwriter'

#### **BigQuery**

In [ ]:
# Proyecto y cliente
project_id='bi-fcom-drmb-local-pe-sbx'
client = bigquery.Client(project=project_id)

# Configuración job
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = (bigquery.ScalarQueryParameter("limit", "INT64", 1000),)

query = """
    SELECT * FROM `bi-fcom-drmb-local-pe-sbx.Dragonite_SX_KPIs.reporte_backlog`
"""

# Query execution
query_job = client.query(query, job_config=job_config)
query_job.result()
data = query_job.to_dataframe()

#### **Indicar usuario**
IMPORTANTE: Solo se debe modificar el usuario, ruta (en caso sea diferente) y fechas de reporte. Considerar que se debe sincronizar la carpeta 'MARKETPLACE PERÚ' antes de realizar la ejecución del script.

In [1]:
usuario = r'ynvillanuevaf'
ruta = r'MARKETPLACE PERÚ - Documentos'

fecha_inicio = '2023-03-20' # Fecha Lunes
fecha_fin = '2023-03-26' # Fecha Domingo

reporte_semanal_sellers = 'no' # Cambiar esto a 'yes' cuando se va procesar los reportes de liquidaciones semanales para los sellers

#### **Importar librerías**

In [2]:
# Importar librerías de funciones
import os # Para trabajar con rutas
import glob # Para jalar todo los archivos en una carpeta
import warnings # Para evitar que salgan errores en formatos de archivo (no altera el producto)
import numpy as np # Para operaciones matemáticas
import pandas as pd # Para transformar la data
from pandas import ExcelWriter # Para exportar tabla a Excel
from rich import print # Usado para definir función nueva 
from xlsx2csv import Xlsx2csv # Usado para definir función nueva
from io import StringIO # Usado para definir función nueva
import xlsxwriter # Funcionalidad para trabajar con archivos Excel
import datetime as dt # Para fechas (opcional)
from datetime import datetime, timedelta  # Para fechas (opcional)
from fpdf import FPDF # Para crear archivos PDF

# Opciones
warnings.filterwarnings(action='ignore') # Eliminar mensajes de warning (no elimina mensajes de error)
pd.set_option('display.expand_frame_repr', False)
pd.options.display.float_format = '{:.2f}'.format # Suprimir notación científica para números

# Fechas
mes_cierre = pd.to_datetime('2022-07-01')
fecha_coe = pd.to_datetime('2022-04-11')
fecha_comision = pd.to_datetime('2022-06-27')
fecha_comision_2 = pd.to_datetime('2022-08-08')
fecha_cofi = pd.to_datetime('2022-08-22')
fecha_cofi_2 = pd.to_datetime('2023-01-01')
fecha_inicio = pd.to_datetime(fecha_inicio)
fecha_fin = pd.to_datetime(fecha_fin)

# Rutas base
ruta_base = r'C:\\Users\\' + usuario + r'\\Falabella\\' + ruta

ruta_pagos = ruta_base + r'\\1. Postventa\\1. Falabella\\1. Pagos'
ruta_devoluciones = ruta_base + r'\\1. Postventa\\1. Falabella\\2. Devoluciones'
ruta_operaciones = ruta_base + r'\\2. Operaciones'
ruta_soporte = ruta_base + r'\\3. Seller Support'

In [3]:
# Funciones

## Función para leer Excel más rápido

def read_excel(path: str, sheet_name: str) -> pd.DataFrame:
    buffer = StringIO()
    Xlsx2csv(path, outputencoding="utf-8", sheet_name=sheet_name).convert(buffer)
    buffer.seek(0)
    df = pd.read_csv(buffer)
    return df

#### **Leer data**

In [4]:
# 1. SRX

ventas_files = glob.glob(os.path.join(ruta_pagos, "1. SRX", "1. Ventas", "[0-9]*.txt"))
ventas_ = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(ventas_files)):
    x = pd.read_csv(ventas_files[i], sep="|", encoding='latin-1')
    ventas_ = ventas_.append(x)

egresos_files = glob.glob(os.path.join(ruta_pagos, "1. SRX", "2. Egresos", "[0-9]*.txt"))
egresos_ = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(egresos_files)):
    x = pd.read_csv(egresos_files[i], sep="|", encoding='latin-1')
    egresos_ = egresos_.append(x)

In [5]:
# 2. Información de sellers

reporte_inbound_files = glob.glob(os.path.join(ruta_soporte, "1. Inbound", "1. Falabella", "1. Información Seller Comercial", "Reporte*Inbound*.xlsx"))
reporte_inbound_ = pd.DataFrame()  # 1. Reporte Inbound
x = pd.DataFrame()
for i in range(len(reporte_inbound_files)):
    x = pd.read_excel(reporte_inbound_files[i], 'Cuentas Comercial')
    reporte_inbound_ = reporte_inbound_.append(x)

In [6]:
# 3. Comercial

comercial_files = glob.glob(os.path.join(ruta_pagos, "5. Información Externa", "0. Comercial", "[A-Z]*.xlsx"))
kams_ = pd.DataFrame()  ### 1. KAMs
kams_contactos_ = pd.DataFrame()  ### 2. KAMs contactos
x = pd.DataFrame()
y = pd.DataFrame()
for i in range(len(comercial_files)):
    x = pd.read_excel(comercial_files[i], 'Hoja1')
    y = pd.read_excel(comercial_files[i], 'Contactos')
    kams_ = kams_.append(x)
    kams_contactos_ = kams_contactos_.append(y)

comisiones_files = glob.glob(os.path.join(ruta_pagos, "5. Información Externa", "1. Comisiones", "[0-9]*.xlsx"))
comisiones_ = pd.DataFrame()  ### 1. Comisiones
x = pd.DataFrame()
for i in range(len(comisiones_files)):
    x = pd.read_excel(comisiones_files[i], 'COMISIONES')
    comisiones_ = comisiones_.append(x)

In [7]:
# 4. Logística

ddp_files = glob.glob(os.path.join(ruta_pagos, "4. Logística", "0. Despacho Directo", "[1]*.xlsx"))
ddp_ = pd.DataFrame()  # 1. Sellers DDP
x = pd.DataFrame()
for i in range(len(ddp_files)):
    x = pd.read_excel(ddp_files[i], 'Hoja1')
    ddp_ = ddp_.append(x)

cofi_files = glob.glob(os.path.join(ruta_pagos, "4. Logística", "2. Cofinanciamiento Logístico", "[1]*.xlsx"))
cofi_ = pd.DataFrame()  # 1. Cofinanciamiento Logístico
x = pd.DataFrame()
for i in range(len(cofi_files)):
    x = pd.read_excel(cofi_files[i], 'data')
    cofi_ = cofi_.append(x)

In [8]:
# 5. Templates Falabella.com

rec_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "1. Recaudos", "[0-9]*.xlsx"))
template_rec_ = pd.DataFrame() ### 1. Recaudos
x = pd.DataFrame()
for i in range(len(rec_files)):
    x = pd.read_excel(rec_files[i], 'Recaudos')
    template_rec_ = template_rec_.append(x)

liq_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "2. Liquidaciones", "[0-9]*.xlsx"))
template_liq_ = pd.DataFrame() ### 2. Liquidaciones
x = pd.DataFrame()
for i in range(len(liq_files)):
    x = pd.read_excel(liq_files[i], 'Liquidación')
    template_liq_ = template_liq_.append(x)

reg_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "3. Regularizaciones", "1. Regularizaciones Aplicadas", "[A-Z]*.csv"))
template_reg_ = pd.DataFrame() ### 3. Regularizaciones
x = pd.DataFrame()
for i in range(len(reg_files)):
    x = pd.read_csv(reg_files[i], sep=",|;", encoding='latin-1')
    template_reg_ = template_reg_.append(x)

rec_sin_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "4. Rechazos y siniestros", "[0-9]*.xlsx"))
template_rec_sin_ = pd.DataFrame() ### 4. Rechazos y siniestros
x = pd.DataFrame()
for i in range(len(rec_sin_files)):
    x = pd.read_excel(rec_sin_files[i], 'Rechazos')
    template_rec_sin_ = template_rec_sin_.append(x)

rec_sin_noproc_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "4. Rechazos y siniestros", "Registros no procesados","[0-9]*.xlsx")) ### 4. Rechazos y siniestros no procesados
template_rec_sin_noproc_ = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(rec_sin_noproc_files)):
    x = pd.read_excel(rec_sin_noproc_files[i], 'Hoja1')
    template_rec_sin_noproc_ = template_rec_sin_noproc_.append(x)

ajust_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "5. Ajustes manuales", "[0-9]*.xlsx"))
template_ajust_ = pd.DataFrame() ### 5. Ajustes manuales
x = pd.DataFrame()
for i in range(len(ajust_files)):
    x = pd.read_excel(ajust_files[i], 'Ajuste manual')
    template_ajust_ = template_ajust_.append(x)

In [9]:
# 6. COE

abonos_rec_files = glob.glob(os.path.join(ruta_pagos, "0. Reportes", "4. Reportes Falabella.com", "4. Rechazos y siniestros", "Abonos procesados","[0-9]*.xlsx")) ### 4. Abonos procesados
abonos_rec_ = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(abonos_rec_files)):
    x = pd.read_excel(abonos_rec_files[i], 'Hoja1')
    abonos_rec_ = abonos_rec_.append(x)

netsuite_files = glob.glob(os.path.join(ruta_pagos, "6. COE", "1. Netsuite", "[A-Z]*.xlsx"))
pagos_coe_ = pd.DataFrame() ### 1. Histórico de pagos COE
x = pd.DataFrame()
for i in range(len(netsuite_files)):
    x = pd.read_excel(netsuite_files[i], 'ResultadosSistemasFacturasPaga')
    pagos_coe_ = pagos_coe_.append(x)

facturas_files = glob.glob(os.path.join(ruta_pagos, "6. COE", "2. Facturas", "[A-Z]*.xlsx"))
facturas_coe_ = pd.DataFrame()  # 2. Facturas comisión COE
x = pd.DataFrame()
for i in range(len(facturas_files)):
    x = pd.read_excel(facturas_files[i], 'Hoja2')
    facturas_coe_ = facturas_coe_.append(x)

In [10]:
# 7. Devoluciones

reporte_devoluciones_files = glob.glob(os.path.join(ruta_devoluciones, "0. Reportes", "Reporte general de egresos y devoluciones - Falabella.com.xlsx"))
# 1. Reporte general de egresos y devoluciones
reporte_devoluciones_ = pd.DataFrame()
x = pd.DataFrame()
for i in range(len(reporte_devoluciones_files)):
    x = pd.read_excel(reporte_devoluciones_files[i], 'data_egresos')
    reporte_devoluciones_ = reporte_devoluciones_.append(x)

#### **Copiar tablas y eliminar inconsistencias de nomenclatura**
Si todo lo que sigue sale mal regresar a este punto. En este proceso se copian las tablas y se eliminan espacios y/o caracteres extraños de las cabeceras de las columnas.

In [11]:
# 1. Información básica

ventas = ventas_.rename(columns=lambda x: x.strip()).copy()
egresos = egresos_.rename(columns=lambda x: x.strip()).copy()

# 2. Información de sellers

reporte_inbound = reporte_inbound_.rename(columns=lambda x: x.strip()).copy()

# 3. Información comercial

kams = kams_.rename(columns=lambda x: x.strip()).copy()
kams_contactos = kams_contactos_.rename(columns=lambda x: x.strip()).copy()
comisiones = comisiones_.rename(columns=lambda x: x.strip()).copy()

# 4. Información de logística

ddp = ddp_.rename(columns=lambda x: x.strip()).copy()
cofi = cofi_.rename(columns=lambda x: x.strip()).copy()

# 5. Información de reporte de liquidaciones

template_rec = template_rec_.rename(columns=lambda x: x.strip()).copy()
template_liq = template_liq_.rename(columns=lambda x: x.strip()).copy()
template_reg = template_reg_.rename(columns=lambda x: x.strip()).copy()
template_rec_sin = template_rec_sin_.rename(columns=lambda x: x.strip()).copy()
template_rec_sin_noproc = template_rec_sin_noproc_.rename(columns=lambda x: x.strip()).copy()
template_ajust = template_ajust_.rename(columns=lambda x: x.strip()).copy()

# 6. COE

abonos_rec = abonos_rec_.rename(columns=lambda x: x.strip()).copy()
facturas_coe = facturas_coe_.rename(columns=lambda x: x.strip()).copy()
pagos_coe = pagos_coe_.rename(columns=lambda x: x.strip()).copy()

# 7. Devoluciones

reporte_devoluciones = reporte_devoluciones_.rename(columns=lambda x: x.strip()).copy()

#### **Modificar tablas de origen**

In [12]:
ventas = ventas_.rename(columns=lambda x: x.strip()).copy()
egresos = egresos_.rename(columns=lambda x: x.strip()).copy()

In [13]:
# 1. SRX

## 1. Ventas

ventas = ventas \
            .query("Item.notnull() & Seller.notnull()") \
            .rename(columns={'RUC seller': 'Data RUC seller', 'Seller': 'Data Seller', 'Nombre/Razon Social Cliente': 'Cliente'})

cols = ['Data Seller', 'Data RUC seller', 'Folio Relacionado', 'Producto (SKU)', 'OC', 'Tienda', 'Caja', 'Trx', 'Local Despacho', 'DNI/RUC Cliente']
ventas[cols] = ventas[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

ventas = ventas \
            .assign(**{
                'Monto Total': lambda x: pd.to_numeric(x['Monto Total'], errors='coerce').replace(np.nan, 0),
                'Monto Dscto': lambda x: pd.to_numeric(x['Monto Dscto'], errors='coerce').replace(np.nan, 0),
                'Monto_Rebaja': lambda x: pd.to_numeric(x['Monto_Rebaja'], errors='coerce').replace(np.nan, 0)
            }) \
            .assign(**{
                'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                'Estado Folio': lambda x: x['Estado Folio'].str.strip(),
                'Ventas': lambda x: x['Monto Total'] - x['Monto Dscto'] - x['Monto_Rebaja'],
                'Cantidad': lambda x: x['Monto Total'] / x['Precio Unitario'],
                'Cliente': lambda x: x['Cliente'].str.replace('-|,|–', '').str.strip().str.title()
            }) \
            .drop(['Item'], axis=1).drop_duplicates()

## 2. Egresos

egresos = egresos \
            .query('Seller!="Seller"') \
            .assign(**{'RUC seller': lambda x: x['Seller'].str.extract('(^[0-9]{1,11})')}) \
            .rename(columns={'SKU': 'Producto (SKU)', 'Referencia OC': 'Folio Relacionado', 'RUC seller': 'Data RUC seller', 'Seller': 'Data Seller',
                            'Código de Local': 'Tienda Egreso', 'Caja de Egreso': 'Caja Egreso', 'Numero de transaccion Egreso': 'Trx Egreso',
                            'Local Trx Orig': 'Tienda', 'Caja Trx Orig': 'Caja', 'Sec Trx Original': 'Trx'})

cols = ['Data Seller', 'Data RUC seller', 'Folio Relacionado', 'Producto (SKU)', 'OC', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 'SKU Egreso', 'Tienda', 'Caja', 'Trx']
egresos[cols] = egresos[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

egresos = egresos \
            .assign(**{ 
                'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], format='%d/%m/%Y', errors='coerce').dt.date,
                'Fecha Trx Orig': lambda x: pd.to_datetime(x['Fecha Trx Orig'].str.title(), format='%d-%b-%y', errors='coerce').dt.date,
                'Estado Folio': lambda x: x['Estado Folio'].str.strip(),
                'Data Seller': lambda x: x['Data Seller'].str.extract('(?<=^[0-9]{10}-)(.*)'),
                'Monto total Egreso': lambda x: x['Monto total Egreso'].replace(np.nan, 0).astype(float)
            }) \
            .drop(['Item', 'Cantidad'], axis=1).drop_duplicates()

In [14]:
# 2. Información de sellers

reporte_inbound = reporte_inbound  \
                    .rename(columns={'RUT_EMPRESA(11)': 'RUT_EMPRESA', reporte_inbound.columns[15]: 'ESTADO SELLER'})
reporte_inbound.columns = reporte_inbound.columns.str.replace('_', ' ')
reporte_inbound = reporte_inbound \
                    .assign(**{
                        'ESTATUS SAP': 'EXTENDIDO',
                        'RUT EMPRESA': lambda x: x['RUT EMPRESA'].astype(str).str.replace('(-)', '')
                    }) \
                    .rename(columns={'RUT EMPRESA': 'RUC seller', 'RAZON SOCIAL': 'Seller', 'ESTATUS SAP': 'STATUS2'}) \
                    .assign(**{
                        'RUC seller': lambda x: x['RUC seller'].astype(str).str.replace('(\.0$)', ''),
                        'N ID': lambda x: x['N ID'].astype(str).str.replace('(\.0$)', ''),
                        'RUC SRX': lambda x: x['RUC seller'].str[:10]
                    }) \
                    .drop_duplicates(['RUC seller'], keep='last')

info_seller = reporte_inbound.drop_duplicates(['N ID'], keep='last').loc[:, ['RUC seller', 'N ID', 'Seller', 'KAM', 'RUC SRX']].drop_duplicates()
info_seller_q = info_seller.loc[:, ['RUC seller', 'N ID']].drop_duplicates()

In [15]:
# 3. Información comercial

kams = kams.rename(columns={'Encargado': 'KAM'}).query("`Categoría`.notnull()").drop(['Etiquetas de fila'], axis=1) \
            .assign(**{'len': lambda x: x['Categoría'].str.len()})

comisiones = comisiones \
                .rename(columns={'MKP_VPC_TECH_KEY': 'RUC seller', 'COMISION': 'Comisión %', comisiones.columns[3]: 'concat'}) \
                .assign(**{
                    'regla': lambda x: np.select([
                                        (x['MKP_JERARQUIA'] == 'JTODO'),
                                        (x['concat'].str.contains('^20[0-9]{1,9}..-.J[0-9]{6}$')),
                                        (x['concat'].str.contains('^0.-.J[0-9]{6}$')),
                                        (x['concat'].str.contains('^20[0-9]{1,9}.-.J[0-9]{4}$')) | (x['concat'].str.contains('^10[0-9]{1,9}.-.J[0-9]{4}$')),
                                        (x['concat'].str.contains('^20[0-9]{1,9}.-.J[0-9]{2}$')) | (x['concat'].str.contains('^10[0-9]{1,9}.-.J[0-9]{2}$')),
                                        (x['concat'].str.contains('^0.-.J[0-9]{4}$'))
                                        ],
                                        ['1', '2', '3', '4', '5', '6'],
                                        default='0'),
                    'RUC seller': lambda x: x['RUC seller'].astype(str).str.replace('(\.0$)', '')
                })

In [16]:
# 4. Información de logística

ddp = ddp.loc[:, ['FACILITY_A']].rename(columns={'FACILITY_A': 'Data RUC seller'}) \
        .assign(**{
            'DDP': 'SI', 
            'Data RUC seller': lambda x: x['Data RUC seller'].astype(str).str.replace('(\.0$)', '')
        })

cofi = cofi \
        .assign(**{'Producto (SKU)': lambda x: x['Producto (SKU)'].astype(str).str.replace('(\.0$)', '')})\
        .drop_duplicates(['Producto (SKU)'], keep='first')

In [17]:
# 5. Información de reporte de liquidaciones

template_rec = template_rec \
                    .assign(**{
                        'Fecha Transacción': lambda x: pd.to_datetime(x['Fecha Transacción'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], infer_datetime_format=True, errors='coerce').dt.date
                    }) \
                    .query("`Fecha Transacción`.notnull()")

cols = ['Semana Recaudación', 'Numero Orden de Venta', 'Folio Relacionado', 'SKU', 'ID SELLER SVL']
template_rec[cols] = template_rec[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

template_liq = template_liq \
                    .assign(**{
                        'Fecha Transacción': lambda x: pd.to_datetime(x['Fecha Transacción'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Pago', 'Descuento')
                    }) \
                    .query("`Fecha Transacción`.notnull()")

cols = ['Semana Recaudación', 'Número Orden de Venta', 'Folio Relacionado', 'SKU', 'ID SELLER SVL']
template_liq[cols] = template_liq[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

template_reg = template_reg \
                    .assign(**{'Semana Recaudación': lambda x: x['FACTURA N.º'].str.extract('([0-9]{8}-[0-9]{8})')}) \
                    .assign(**{
                        'Semana Recaudación': lambda x: np.select([
                                                        (x['Semana Recaudación'] == '20220601-20220605'),
                                                        (x['Semana Recaudación'] == '20220911-20220915')
                                                        ],
                                                        ['20220523-20220529', '20220905-20220911'], 
                                                        default=x['Semana Recaudación']),
                        'ID SELLER SVL': lambda x: x['FACTURA N.º'].str.extract('(-[0-9]{4,5}-)').apply(lambda x: x.str.replace('-', ''))
                    }) \
                    .assign(**{
                        'Semana Recaudación': lambda x: x['Semana Recaudación'].astype(str).str.replace('(\.0$)', ''),
                        'ID SELLER SVL': lambda x: x['ID SELLER SVL'].astype(str).str.replace('(\.0$)', '')
                    }) \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'IMPORTE']].rename(columns={'IMPORTE': 'Importe a Pagar'}) \
                    .assign(**{
                        'Importe a Pagar': lambda x: x['Importe a Pagar']*-1,
                        'Tipo': 'Regularización'
                    })

template_rec_sin = template_rec_sin \
                        .rename(columns={'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)'}) \
                        .query("`Folio Relacionado`.notnull()") \
                        .loc[:, ['Semana Recaudación', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'ID SELLER SVL', 'Cantidad', 'Unit Price', 'Shipping',
                                 'Descuento', 'Monto Total', 'Importe Comisión', 'Importe a Pagar', '% Comisión', 'Serie', 'Factura', 'Tipo de Transaccion']]

cols = ['OC', 'Folio Relacionado', 'Producto (SKU)', 'ID SELLER SVL', 'Serie', 'Factura']
template_rec_sin[cols] = template_rec_sin[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

template_rec_sin = template_rec_sin.drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

template_rec_sin_noproc = template_rec_sin_noproc \
                                .rename(columns={'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)'}) \
                                .loc[:, ['Semana Recaudación', 'OC', 'Folio Relacionado', 'Producto (SKU)']] \
                                .assign(**{'Estado Rechazo': 'NO PROCESADO'})

cols = ['OC', 'Folio Relacionado', 'Producto (SKU)']
template_rec_sin_noproc[cols] = template_rec_sin_noproc[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

template_rec_sin_noproc = template_rec_sin_noproc.drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

template_rec_sin = template_rec_sin \
                    .merge(template_rec_sin_noproc, how='left', on=['Semana Recaudación', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .query("`Estado Rechazo` != 'NO PROCESADO' & `OC`!='nan'") \
                    .drop(['Estado Rechazo'], axis=1) \
                    .assign(**{'Tipo': 'Devoluciones'})

template_ajust = template_ajust \
                    .rename(columns={'Semana Recaudo': 'Semana Recaudación'}) \
                    .assign(**{
                        'Fecha Transacción': lambda x: pd.to_datetime(x['Fecha Transacción'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Tipo': 'Ajuste'
                    }) \
                    .query("`Tipo de Transaccion`.notnull()")

cols = ['Semana Recaudación', 'Número Orden de Venta', 'Folio Relacionado', 'SKU', 'ID SELLER SVL']
template_ajust[cols] = template_ajust[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))


In [18]:
# 6. COE

abonos_rec = abonos_rec \
                .rename(columns={'Fecha de vencimiento/Fecha límite de recepción': 'Fecha de Pago'}) \
                .assign(**{
                        'Fecha de Pago': lambda x: pd.to_datetime(x['Fecha de Pago'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Semana Recaudación': lambda x: x['Número de documento'].str.extract('([0-9]{8}-[0-9]{8})'),
                        'ID SELLER SVL': lambda x: x['Número de documento'].str.extract('([0-9]{1,6})'),
                        'Importe': lambda x: round(x['Importe restante'], 2),
                        'Documento': 'FACTURA DEVOLUCIONES'
                        }) \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Número de documento', 'Importe', 'Fecha de Pago', 'Documento']]

pagos_coe = pagos_coe \
                .assign(**{
                    'Fecha Factura': lambda x: pd.to_datetime(x['Fecha Factura'], infer_datetime_format=True, errors='coerce').dt.date,
                    'Fecha de Pago': lambda x: pd.to_datetime(x['Fecha de Pago'], infer_datetime_format=True, errors='coerce').dt.date,
                    'Semana Recaudación': lambda x: x['Factura'].str.extract('([0-9]{8}-[0-9]{8})'),
                    'ID SELLER SVL': lambda x: x['Factura'].str.extract('(-[0-9]{4,5}-)').apply(lambda x: x.str.replace('-', ''))
                }) \
                .assign(**{
                    'Semana Recaudación': lambda x: x['Semana Recaudación'].astype(str).str.replace('(\.0$)', ''),
                    'ID SELLER SVL': lambda x: x['ID SELLER SVL'].astype(str).str.replace('(\.0$)', '')
                }) \
                .query("`Semana Recaudación`.notnull()").query("`Semana Recaudación`!='nan'") \
                .drop_duplicates(['Semana Recaudación', 'ID SELLER SVL', 'Factura', 'Fecha Factura', 'Importe Factura', 'Importe Pago'], keep='last')

facturas_coe = facturas_coe.query("Corte.notnull()") \
                    .assign(**{
                        'Corte': lambda x: x['Corte'].astype(str).str.replace('(\.0$)', ''),
                        'Tercero': lambda x: x['Tercero'].astype(str).str.replace('(\.0$)', '')
                    }) \
                    .rename(columns={'Corte': 'Semana Recaudación', 'No. Factura': 'Factura comisión', 'Valor factura': 'Importe factura comisión',
                                            'No. Nota credito': 'Nota de crédito comisión', 'Valor NC': 'Importe nota de crédito comisión'}) \
                    .assign(**{
                                'ID SELLER SVL': lambda x: x['Tercero'].str.extract('([0-9]{1,6})'),
                                'Importe factura comisión': lambda x: round(x['Importe factura comisión'] * 1.18, 2),
                                'Importe nota de crédito comisión': lambda x: round(x['Importe nota de crédito comisión'] * 1.18, 2)
                            }) \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Factura comisión', 'Importe factura comisión', 'Nota de crédito comisión', 'Importe nota de crédito comisión']]

In [19]:
# 7. Devoluciones

reporte_devoluciones = reporte_devoluciones \
                            .assign(**{
                                'Fecha de Retorno': lambda x: pd.to_datetime(x['Fecha de Retorno'], infer_datetime_format=True, errors='coerce').dt.date
                            }) \
                            .loc[:, ['OC','Folio Relacionado', 'Producto (SKU)', 
                                                    'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción',
                                                    'Distrito', 'Provincia',
                                                    'Despacho Directo', 'Status Recojo', 'Estado Egreso',
                                                    'Encargado Egreso', 'Transporte', 'F3', 'Guia de Retorno',
                                                    'Fecha de Retorno', 'Motivo Devolución', 'Dirección Entrega',
                                                    'Persona Entrega', 'Link Guia', 'Comisión %', 'Estado Devolución', 
                                                    'Usuario', 'CODIGO_PERFIL'
                                                    ]]

cols = ['OC', 'Folio Relacionado', 'Producto (SKU)', 'F3', 'Guia de Retorno']
reporte_devoluciones[cols] = reporte_devoluciones[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

#### **Estandarizar datos de sellers**

In [20]:
# 1. Lista general de sellers

lista_sellers = pd.concat([
                        ventas[['Data RUC seller', 'Data Seller']].drop_duplicates(), 
                        egresos[['Data RUC seller', 'Data Seller']].drop_duplicates()
                        ], axis=0).drop_duplicates() \
                    .sort_values(['Data RUC seller', 'Data Seller'], ascending=False) \
                    .assign(**{'len': lambda x: x['Data RUC seller'].str.len()}) \
                    .sort_values(['len'], ascending=False).query("`len`>0")

# 2. Cruce RUCs

lista_sellers_ruc11 = lista_sellers.query("`len`==11").drop(['len'], axis=1)

lista_sellers_ruc10 = lista_sellers.query("`len`==10").drop(['len'], axis=1) \
                        .merge(info_seller[['RUC SRX', 'RUC seller', 'Seller']].drop_duplicates(['RUC SRX'], keep='first'), 
                            how='left', left_on='Data RUC seller', right_on='RUC SRX') \
                        .drop(['RUC SRX'], axis=1)

lista_sellers_otros = lista_sellers.query("`len`<10").drop(['len'], axis=1) \
                        .assign(**{'Data Seller': lambda x: x['Data Seller'].str.replace('\.', '')}) \
                        .merge(info_seller[['RUC seller', 'Seller']].drop_duplicates(['RUC seller'], keep='first'), 
                            how='left', left_on='Data Seller', right_on='Seller')

# 3. Lista general de sellers para cruce

lista_sellers = pd.concat([lista_sellers_ruc11, lista_sellers_ruc10, lista_sellers_otros], axis=0) \
                    .assign(**{
                        'RUC seller': lambda x: np.select([(x['RUC seller'].isnull() | x['RUC seller'].isna()), (x['RUC seller'].notnull())], 
                                                        [x['Data RUC seller'], x['RUC seller']],
                                                        default='NA'),
                        'Seller': lambda x: np.select([(x['Seller'].isnull() | x['Seller'].isna()), (x['Seller'].notnull())], 
                                                        [x['Data Seller'], x['Seller']],
                                                        default='NA')                        
                    }) \
                    .sort_values(['RUC seller', 'Seller'], ascending=True) \
                    .drop(['Seller'], axis=1)

nombre_sellers = reporte_inbound \
                    .loc[:, ['RUC seller', 'Seller']] \
                    .assign(**{'Seller': lambda x: x['Seller'].str.upper().str.replace('(\/)', '')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.upper()}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('"', '')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('´', '')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace(',', '')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('.', '')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('¨', '')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('–', '-')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('?', '-')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ñ', 'N')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Á', 'A')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('É', 'E')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Í', 'I')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ó', 'O')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ú', 'U')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ä', 'A')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ë', 'E')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ï', 'I')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ö', 'O')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('Ü', 'U')}) \
                    .assign(**{'Seller': lambda x: x['Seller'].str.replace('  ', ' ')}) \
                    .assign(**{'Seller': lambda x: np.where((x['RUC seller'] == '20547836473'), 'FALABELLA.COM S.A.C.', x['Seller'])})    

lista_sellers = pd.merge(lista_sellers, nombre_sellers, how='left', on='RUC seller')

# 5. Cruzar lista de sellers con las tablas

ventas = pd.merge(ventas, lista_sellers, how='left', on=['Data RUC seller', 'Data Seller'])
egresos = pd.merge(egresos, lista_sellers, how='left', on=['Data RUC seller', 'Data Seller'])

In [21]:
ddp = pd.merge(ddp, lista_sellers, how='left', on=['Data RUC seller']).loc[:, ['RUC seller']].query("`RUC seller`.notnull()")

sellers_ddp = ddp['RUC seller'].to_list()

#### **Tablas auxiliares**

In [22]:
# Tablas auxiliares (DEMORA MUCHO!!)

## 1. Resumen de ventas
ventas_rr = ventas[['OC', 'Folio Relacionado', 'Producto (SKU)']].drop_duplicates()

## 10. Hallar cantidad de productos por combinación 'OC', 'Folio Relacionado' (solo se debe usar los que n=1)
n = ventas_rr.groupby(['OC', 'Folio Relacionado'], dropna=False)['Producto (SKU)'].count().reset_index().rename(columns={'Producto (SKU)': 'n'})

ventas_n = pd.merge(ventas_rr, n, how='left', on=['OC', 'Folio Relacionado'])
ventas_n1 = ventas_n[ventas_n['n']==1]

## 11. Hallar cantidad de productos por combinación 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Monto Total' (solo se debe usar los que n=1)
ventas_rrm = ventas[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Monto Total']].drop_duplicates()
o = ventas_rrm.groupby(['OC', 'Folio Relacionado', 'Monto Total'], dropna=False)['Producto (SKU)'].count().reset_index().rename(columns={'Producto (SKU)': 'n'})

ventas_o = pd.merge(ventas_rrm, o, how='left', on=['OC', 'Folio Relacionado', 'Monto Total'])
ventas_o1 = ventas_o[ventas_o['n']==1]

## 12. Hallar cantidad de productos por combinación 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Precio Unitario' (solo se debe usar los que n=1)
ventas_rrpu = ventas[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Precio Unitario']].drop_duplicates()
pu = ventas_rrpu.groupby(['OC', 'Folio Relacionado', 'Precio Unitario'], dropna=False)['Producto (SKU)'].count().reset_index().rename(columns={'Producto (SKU)': 'n'})

ventas_pu = ventas_rrpu.merge(pu, how='left', on=['OC', 'Folio Relacionado', 'Precio Unitario'])
ventas_pu1 = ventas_pu[ventas_pu['n']==1]

## 14. Hallar cantidad de categorías por RUC seller
q = ventas.loc[:, ['RUC seller', 'Cod.Subclase']].drop_duplicates().groupby(['RUC seller'], dropna=False)['Cod.Subclase'].count().reset_index().rename(columns={'Cod.Subclase': 'q'})

ventas_q1 = ventas \
                .loc[:, ['RUC seller', 'Cod.Subclase']].drop_duplicates() \
                .merge(q, how='left', on=['RUC seller']) \
                .query("q==1")

## 16. Hallar cantidad de productos por combinación 'Fecha de Compra', 'Tienda', 'Caja', 'Trx'
s = ventas \
        .groupby(['Fecha de Compra', 'Tienda', 'Caja', 'Trx'], dropna=False)['Producto (SKU)'].count().reset_index().rename(columns={'Producto (SKU)': 's'})

ventas_s1 = ventas \
            .loc[:, ['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx', 'Precio Unitario', 'Monto Total', 'Cod.Subclase']].drop_duplicates() \
            .merge(s, how='left', on=['Fecha de Compra', 'Tienda', 'Caja', 'Trx']) \
            .query("s==1")
ventas_sz = ventas_s1.loc[:, ['Fecha de Compra', 'Tienda', 'Caja', 'Trx', 's']].drop_duplicates()

## 19. Auxiliar para recaudos
ventas_rec = ventas[['OC', 'Producto (SKU)', 'Folio Relacionado', 'RUC seller', 'Seller', 'Linea', 'Sublinea', 'Clase', 'Subclase', 'Cod.Subclase', 'Descripción', 'Distrito', 'Provincia', 'DNI/RUC Cliente', 'Cliente']].drop_duplicates(['OC', 'Producto (SKU)'], keep='first')

## 20. Auxiliar para liquidaciones
ventas_liq = ventas[['OC', 'Producto (SKU)', 'Folio Relacionado', 'RUC seller', 'Seller', 'Linea', 'Sublinea', 'Clase', 'Subclase', 'Cod.Subclase', 'Descripción', 'Distrito', 'Provincia', 'DNI/RUC Cliente', 'Cliente']].drop_duplicates(['OC', 'Producto (SKU)'], keep='first')

ventas_rt = ventas[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Linea', 'Sublinea',
                    'Clase', 'Subclase', 'Local Despacho', 'Descripción']].drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

egresos_rt = egresos[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso']].drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

#### Resumen de ventas
ventas_r = ventas[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Monto Total', 'Monto Dscto', 'Monto_Rebaja', 'Ventas', 'Local Despacho']]

ventas_rr_ = ventas_rr.rename(columns={'OC': 'Nuevo OC', 'Producto (SKU)': 'Nuevo Producto (SKU)', 'Folio Relacionado': 'Nuevo Folio Relacionado'})

#### **Estandarización de egresos**

In [23]:
# 1. Transformación de data

egresos = egresos \
            .merge(ventas.loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'Fecha de Compra', 'Precio Unitario', 'Monto Total', 'Cod.Subclase']].drop_duplicates(), how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
            .assign(**{
                'Fecha de Compra': lambda x: np.where(((x['Fecha de Compra'].isnull()) & (x['Fecha Trx Orig'].notnull())), x['Fecha Trx Orig'], x['Fecha de Compra'])
            })

# 2. Transformación de data ok y no ok

## A. Data OK (Arreglar RUC seller y Seller)

egresos_ok = egresos.query("`Monto Total`.notnull()") \
                .drop(['RUC seller', 'Seller'], axis=1) \
                .merge(ventas.loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'RUC seller', 'Seller']].drop_duplicates(), 
                        how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)'])

## B. Data No OK (Unir con tabla auxiliar para determinar si se puede arreglar o no la data)

egresos_nok = egresos.query("`Monto Total`.isnull()") \
                .merge(ventas_sz, how='left', on=['Fecha de Compra', 'Tienda', 'Caja', 'Trx'])

egresos_nok_1a = egresos_nok.query("`s`.notnull()") \
                    .drop(['RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Precio Unitario', 'Monto Total', 'Cod.Subclase'], axis=1) \
                    .merge(ventas_s1, how='left', on=['Fecha de Compra', 'Tienda', 'Caja', 'Trx', 's'])

egresos_nok_1b = egresos_nok.query("`s`.isnull()")

## C. Ajustar Data OK y No OK

egresos_ok = pd.concat([egresos_ok, egresos_nok_1a], axis=0)
egresos_nok = egresos_nok_1b
egresos = pd.concat([egresos_ok, egresos_nok], axis=0).drop(['Monto Total', 's'], axis=1)

# 3. Condicional data correcta e incorrecta

## A. PRIMER PROCESAMIENTO

### 1. Reglas

egresos = egresos \
            .assign(**{
                'Correcto': lambda x: np.where(
                    (x['Fecha de Compra'].notnull()) & 
                    (x['OC'].str.contains('^5')) & (x['OC'].str.len() == 10) &
                    (x['Folio Relacionado'].str.contains('^1')) & (x['Folio Relacionado'].str.len() >= 11) & (x['Folio Relacionado'].str.len() <= 12) & (x['Producto (SKU)'].str.contains('^1')) & (x['Producto (SKU)'].str.len() == 8)
                    , 'Si', 'No')
            })

egresos_ok = egresos.query("Correcto=='Si'").drop(['Correcto'], axis=1)
egresos_nok = egresos.query("Correcto=='No'").drop(['Correcto'], axis=1)
egresos = egresos.drop(['Correcto'], axis=1)

### 2. Encontrar Folio Relacionado

egresos_nok = egresos_nok \
                .assign(**{
                    'Nuevo OC': lambda x: np.select(
                                            [
                                                (x['OC'].str.contains('^5')) & (x['OC'].str.len() == 10),
                                                (x['Folio Relacionado'].str.contains('^5')) & (x['Folio Relacionado'].str.len() == 10),
                                                (x['Producto (SKU)'].str.len() == 9)
                                            ],
                                            [x['OC'], x['Folio Relacionado'], '5' + x['Producto (SKU)']],
                                            default=np.nan
                    ),
                    'Nuevo Producto (SKU)': lambda x: np.select(
                                            [
                                                (x['OC'].str.contains('^1')) & (x['OC'].str.len() == 8),
                                                (x['Folio Relacionado'].str.contains('^1')) & (x['Folio Relacionado'].str.len() == 8),
                                                (x['Producto (SKU)'].str.contains('^1')) & (x['Producto (SKU)'].str.len() == 8)
                                            ],
                                            [x['OC'], x['Folio Relacionado'], x['Producto (SKU)']],
                                            default=np.nan
                    )
                }) \
                .merge(ventas_rr_, how='left', on=['Nuevo OC', 'Nuevo Producto (SKU)'])

egresos_nok1 = egresos_nok.query("`Nuevo Folio Relacionado`.notnull()") \
                    .loc[:, ['Fecha de Egreso', 'Tienda Egreso', 'Nombre de Local', 'SKU Egreso',
                             'EAN Egreso', 'Data Seller', 'Caja Egreso', 'Trx Egreso',
                             'Medio pago Egreso', 'Cod.vendedor', 'Vendedor', 'Nuevo Producto (SKU)',
                             'Jerarquia', 'Nuevo OC', 'Tipo Producto', 'Medio Pago Original',
                             'Monto total Egreso', 'Nuevo Folio Relacionado', 'Estado Folio',
                             'Fecha Trx Orig', 'Tienda', 'Caja', 'Trx', 'Monto Original',
                             'Tarjeta Original', 'CodigoBan', 'Data RUC seller', 'Fecha de Compra',
                             'Precio Unitario', 'Cod.Subclase', 'RUC seller', 'Seller']] \
                    .rename(columns={'Nuevo OC': 'OC', 'Nuevo Producto (SKU)': 'Producto (SKU)', 'Nuevo Folio Relacionado': 'Folio Relacionado'})

egresos_nok2 = egresos_nok.query("`Nuevo Folio Relacionado`.isnull()") \
                    .drop(['Nuevo Folio Relacionado'], axis=1) \
                    .assign(**{
                        'Nuevo Folio Relacionado': lambda x: np.select(
                            [
                            (x['OC'].str.contains('^1')) & (x['OC'].str.len() >= 11) & (x['OC'].str.len() <= 12),
                            (x['Folio Relacionado'].str.contains('^1')) & (x['Folio Relacionado'].str.len() >= 11) & (x['Folio Relacionado'].str.len() <= 12),
                            (x['Producto (SKU)'].str.contains('^1')) & (x['Producto (SKU)'].str.len() >= 11) & (x['Producto (SKU)'].str.len() <= 12)
                            ],[
                            x['OC'], x['Folio Relacionado'], '5' + x['Producto (SKU)']
                            ],
                            default=np.nan
                        )
                    }) \
                    .assign(**{
                        'OC': lambda x: np.where((x['Nuevo OC'].notnull()), x['Nuevo OC'], x['OC']),
                        'Folio Relacionado': lambda x: np.where((x['Nuevo Folio Relacionado'].notnull()), x['Nuevo Folio Relacionado'], x['Folio Relacionado']),
                        'Producto (SKU)': lambda x: np.where((x['Nuevo Producto (SKU)'].notnull()), x['Nuevo Producto (SKU)'], x['Producto (SKU)'])
                    }) \
                    .drop(['Nuevo Folio Relacionado', 'Nuevo Producto (SKU)'], axis=1) \
                    .assign(**{'Regla': lambda x: np.where((x['OC'].str.contains('^5')) & (x['OC'].str.len() == 10), '1', '2')})

egresos_nok_1 = egresos_nok2.query("Regla=='1'").drop(['Regla'], axis=1) 
egresos_nok_2 = egresos_nok2.query("Regla=='2'").drop(['Regla'], axis=1) 

### 3. Regla 1

#### A. Cruce Monto Total

##### 1. Data Cruce
ventas_o1_ = ventas_o1.rename(columns={'Producto (SKU)': 'Nuevo Producto (SKU)', 'Folio Relacionado': 'Nuevo Folio Relacionado'}) \
                        .assign(**{'Monto total Egreso': lambda x: x['Monto Total'] * -1}) \
                        .drop(['Monto Total', 'n'], axis=1).drop_duplicates(['OC', 'Monto total Egreso'], keep='first')

##### 2. Unir tabla

egresos_nok_1 = pd.merge(egresos_nok_1, ventas_o1_, how='left', on=['OC', 'Monto total Egreso'])

##### 3. Ordenar data ok y no ok

egresos_nok_11 = egresos_nok_1.query("`Nuevo Producto (SKU)`.notnull()") \
                    .loc[:, ['Fecha de Egreso', 'Tienda Egreso', 'Nombre de Local', 'SKU Egreso',
                             'EAN Egreso', 'Data Seller', 'Caja Egreso', 'Trx Egreso',
                             'Medio pago Egreso', 'Cod.vendedor', 'Vendedor', 'Nuevo Producto (SKU)',
                             'Jerarquia', 'OC', 'Tipo Producto', 'Medio Pago Original',
                             'Monto total Egreso', 'Nuevo Folio Relacionado', 'Estado Folio',
                             'Fecha Trx Orig', 'Tienda', 'Caja', 'Trx', 'Monto Original',
                             'Tarjeta Original', 'CodigoBan', 'Data RUC seller', 'Fecha de Compra',
                             'Precio Unitario', 'Cod.Subclase', 'RUC seller', 'Seller']] \
                    .rename(columns={'Nuevo Producto (SKU)': 'Producto (SKU)', 'Nuevo Folio Relacionado': 'Folio Relacionado'})

egresos_nok_12 = egresos_nok_1.query("`Nuevo Producto (SKU)`.isnull()") \
                    .drop(['Nuevo Folio Relacionado', 'Nuevo Producto (SKU)'], axis=1)

#### B. Cruce Precio Unitario

ventas_pu1_ = ventas_pu1.rename(columns={'Producto (SKU)': 'Nuevo Producto (SKU)', 'Folio Relacionado': 'Nuevo Folio Relacionado'}) \
                        .assign(**{'Monto total Egreso': lambda x: x['Precio Unitario'] * -1}) \
                        .drop(['Precio Unitario', 'n'], axis=1).drop_duplicates(['OC', 'Monto total Egreso'], keep='first')

egresos_nok_12 = pd.merge(egresos_nok_12, ventas_pu1_, how='left', on=['OC', 'Monto total Egreso'])

##### 3. Ordenar data ok y no ok

egresos_nok_121 = egresos_nok_12.query("`Nuevo Producto (SKU)`.notnull()") \
                    .loc[:, ['Fecha de Egreso', 'Tienda Egreso', 'Nombre de Local', 'SKU Egreso',
                             'EAN Egreso', 'Data Seller', 'Caja Egreso', 'Trx Egreso',
                             'Medio pago Egreso', 'Cod.vendedor', 'Vendedor', 'Nuevo Producto (SKU)',
                             'Jerarquia', 'OC', 'Tipo Producto', 'Medio Pago Original',
                             'Monto total Egreso', 'Nuevo Folio Relacionado', 'Estado Folio',
                             'Fecha Trx Orig', 'Tienda', 'Caja', 'Trx', 'Monto Original',
                             'Tarjeta Original', 'CodigoBan', 'Data RUC seller', 'Fecha de Compra',
                             'Precio Unitario', 'Cod.Subclase', 'RUC seller', 'Seller']] \
                    .rename(columns={'Nuevo Producto (SKU)': 'Producto (SKU)', 'Nuevo Folio Relacionado': 'Folio Relacionado'})

egresos_nok_122 = egresos_nok_12.query("`Nuevo Producto (SKU)`.isnull()") \
                    .drop(['Nuevo Folio Relacionado', 'Nuevo Producto (SKU)'], axis=1)

##### 4. Unir tablas #2068 (OK)

egresos_ok_2 = pd.concat([egresos_nok_11, egresos_nok_121], axis=0)  # 344 #14
egresos_nok_2 = pd.concat([egresos_nok_2, egresos_nok_122], axis=0)  # 980 #731

### 4. Regla 2

#### A. Cruce Monto Total

##### 1. Data Cruce
ventas_o1_ = ventas_o1.rename(columns={'Producto (SKU)': 'Nuevo Producto (SKU)', 'OC': 'Nuevo OC'}) \
                        .assign(**{'Monto total Egreso': lambda x: x['Monto Total'] * -1}) \
                        .drop(['Monto Total', 'n'], axis=1).drop_duplicates(['Folio Relacionado', 'Monto total Egreso'], keep='first')

##### 2. Unir tabla

egresos_nok_2 = egresos_nok_2.drop(['Nuevo OC'], axis=1).merge(ventas_o1_, how='left', on=['Folio Relacionado', 'Monto total Egreso'])

##### 3. Ordenar data ok y no ok

egresos_nok_21 = egresos_nok_2.query("`Nuevo Producto (SKU)`.notnull()") \
                    .loc[:, ['Fecha de Egreso', 'Tienda Egreso', 'Nombre de Local', 'SKU Egreso',
                               'EAN Egreso', 'Data Seller', 'Caja Egreso', 'Trx Egreso',
                               'Medio pago Egreso', 'Cod.vendedor', 'Vendedor', 'Nuevo Producto (SKU)',
                               'Jerarquia', 'Nuevo OC', 'Tipo Producto', 'Medio Pago Original',
                               'Monto total Egreso', 'Folio Relacionado', 'Estado Folio',
                               'Fecha Trx Orig', 'Tienda', 'Caja', 'Trx', 'Monto Original',
                               'Tarjeta Original', 'CodigoBan', 'Data RUC seller', 'Fecha de Compra',
                               'Precio Unitario', 'Cod.Subclase', 'RUC seller', 'Seller']] \
                    .rename(columns={'Nuevo Producto (SKU)': 'Producto (SKU)', 'Nuevo OC': 'OC'})

egresos_nok_22 = egresos_nok_2.query("`Nuevo Producto (SKU)`.isnull()") \
                    .drop(['Nuevo OC', 'Nuevo Producto (SKU)'], axis=1)

#### B. Cruce Precio Unitario

##### 1. Data Cruce
ventas_pu1_ = ventas_pu1.rename(columns={'Producto (SKU)': 'Nuevo Producto (SKU)', 'OC': 'Nuevo OC'}) \
                        .assign(**{'Monto total Egreso': lambda x: x['Precio Unitario'] * -1}) \
                        .drop(['Precio Unitario', 'n'], axis=1).drop_duplicates(['Folio Relacionado', 'Monto total Egreso'], keep='first')

##### 2. Unir tabla

egresos_nok_22 = pd.merge(egresos_nok_22, ventas_pu1_, how='left', on=['Folio Relacionado', 'Monto total Egreso'])

##### 3. Ordenar data ok y no ok

egresos_nok_221 = egresos_nok_22.query("`Nuevo Producto (SKU)`.notnull()") \
                    .loc[:, ['Fecha de Egreso', 'Tienda Egreso', 'Nombre de Local', 'SKU Egreso',
                             'EAN Egreso', 'Data Seller', 'Caja Egreso', 'Trx Egreso',
                             'Medio pago Egreso', 'Cod.vendedor', 'Vendedor', 'Nuevo Producto (SKU)',
                             'Jerarquia', 'Nuevo OC', 'Tipo Producto', 'Medio Pago Original',
                             'Monto total Egreso', 'Folio Relacionado', 'Estado Folio',
                             'Fecha Trx Orig', 'Tienda', 'Caja', 'Trx', 'Monto Original',
                             'Tarjeta Original', 'CodigoBan', 'Data RUC seller', 'Fecha de Compra',
                             'Precio Unitario', 'Cod.Subclase', 'RUC seller', 'Seller']] \
                    .rename(columns={'Nuevo Producto (SKU)': 'Producto (SKU)', 'Nuevo OC': 'OC'})

egresos_nok_222 = egresos_nok_22.query("`Nuevo Producto (SKU)`.isnull()") \
                    .drop(['Nuevo OC', 'Nuevo Producto (SKU)'], axis=1)

##### 4. Unir tablas #2068 (OK)

egresos_ok_3 = pd.concat([egresos_ok_2, egresos_nok_21, egresos_nok_221], axis=0)
egresos_nok_3 = egresos_nok_222

# 5. Cruce con reglas (para corregir nuevos egresos nok)

### 1. Ventas

ventas_rr_egresos_nok = ventas_rr.copy() \
                            .assign(**{
                                'regla1': lambda x: x['OC'].astype(str).str[2:8],
                                'regla2': lambda x: x['OC'].astype(str).str[0:7],
                                'regla3': lambda x: x['OC'].astype(str).str[4:10],
                                'regla4': lambda x: x['OC'].astype(str).str[-6:]
                            }) \
                            .rename(columns={'OC': 'Nueva OC', 'Folio Relacionado': 'Nuevo Folio Relacionado'})

### 2. Egresos

q = egresos_nok_3 \
        .assign(**{
            'regla1': lambda x: x['OC'].astype(str).str[2:8],
            'regla2': lambda x: x['OC'].astype(str).str[0:7],
            'regla3': lambda x: x['OC'].astype(str).str[4:10],
            'regla4': lambda x: x['OC'].astype(str).str[-6:]
        })

## 2. Unir reglas

z_1 = q.merge(ventas_rr_egresos_nok.drop(['regla2', 'regla3', 'regla4'], axis=1), how='left', on=['regla1', 'Producto (SKU)']).drop_duplicates()

z_2 = z_1.query("`Nueva OC`.isnull()") \
        .drop(['Nueva OC', 'Nuevo Folio Relacionado'], axis=1) \
        .merge(ventas_rr_egresos_nok.drop(['regla1', 'regla3', 'regla4'], axis=1), how='left', on=['regla2', 'Producto (SKU)']).drop_duplicates()

z_3 = z_2.query("`Nueva OC`.isnull()")  \
        .drop(['Nueva OC', 'Nuevo Folio Relacionado'], axis=1) \
        .merge(ventas_rr_egresos_nok.drop(['regla1', 'regla2', 'regla4'], axis=1), how='left', on=['regla3', 'Producto (SKU)']).drop_duplicates()

z_4 = z_3.query("`Nueva OC`.isnull()")  \
        .drop(['Nueva OC', 'Nuevo Folio Relacionado'], axis=1) \
        .merge(ventas_rr_egresos_nok.drop(['regla1', 'regla2', 'regla3'], axis=1), how='left', on=['regla4', 'Producto (SKU)']).drop_duplicates()

z_5 = z_4.query("`Nueva OC`.isnull()")  \
        .drop(['Nueva OC', 'Nuevo Folio Relacionado'], axis=1)

## 3. Separar y arreglar tablas ok y nok

egresos_ok_4 = pd.concat([
                    z_1[z_1['Nueva OC'].notnull()], 
                    z_2[z_2['Nueva OC'].notnull()], 
                    z_3[z_3['Nueva OC'].notnull()], 
                    z_4[z_4['Nueva OC'].notnull()]
                    ], axis=0)  \
                .drop(['OC', 'Folio Relacionado', 'regla1', 'regla2', 'regla3', 'regla4'], axis=1) \
                .rename(columns={'Nueva OC': 'OC', 'Nuevo Folio Relacionado': 'Folio Relacionado'})
egresos_nok_4 = z_5.drop(['regla1', 'regla2', 'regla3', 'regla4'], axis=1)

# 6. Nota Temporal

egresos_ok = pd.concat([egresos_ok, egresos_ok_3, egresos_ok_4], axis=0) \
             .drop(['Fecha de Compra', 'RUC seller', 'Seller', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Precio Unitario'], axis=1) \
             .merge(ventas.loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'RUC seller', 'Seller', 'Fecha de Compra', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Cantidad', 'Precio Unitario', 'Monto Total', 'Cliente', 'DNI/RUC Cliente']].drop_duplicates(), how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
             .assign(**{'Regla': lambda x: np.select([(x['Precio Unitario'] == x['Monto total Egreso'].abs()), 
                                                      (x['Monto Total'] == x['Monto total Egreso'].abs())],
                                                    ['1', '2'], 
                                                    default='3')
             })

egresos_nok = egresos_nok_4

## 2. Verificación de reglas

### 1. Regla 1

egresos_ok1 = egresos_ok.query("Regla=='1'").drop(['Regla'], axis=1) \
                .loc[:, ['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 'Cantidad', 'Precio Unitario', 'Monto total Egreso', 'Cliente', 'DNI/RUC Cliente']]

### 2. Regla 2

egresos_ok2 = egresos_ok.query("Regla=='2'").drop(['Regla'], axis=1) \
                .loc[:, ['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 'Cantidad', 'Precio Unitario', 'Monto total Egreso', 'Cliente', 'DNI/RUC Cliente']]

### 3. Regla 3

egresos_ok3 = egresos_ok.query("Regla=='3'").drop(['Regla'], axis=1) \
                .assign(**{'Regla': lambda x: np.where((x['Monto Total'] >= x['Monto total Egreso'].abs()), '1', '2')})

egresos_ok3 = pd.concat([
                        egresos_ok3.query("Regla=='1'").drop(['Regla'], axis=1), 
                        egresos_ok3.query("Regla=='2'").drop(['Regla'], axis=1).assign(**{'Monto total Egreso': lambda x: x['Monto Total']*-1})
                        ], axis=0) \
                .loc[:, ['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 'Cantidad', 'Precio Unitario', 'Monto total Egreso', 'Cliente', 'DNI/RUC Cliente']]

## 3. Unir todo

egresos_ok = pd.concat([egresos_ok1, egresos_ok2, egresos_ok3], axis=0)

egresos_nok = egresos_nok \
                    .assign(**{
                        'Cantidad': 1,
                        'Precio Unitario': lambda x: x['Monto total Egreso'],
                        'Cod.Subclase': lambda x: x['Jerarquia'].str.extract('(^J[0-9]{1,10})')
                    }) \
                    .loc[:, ['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 'Cantidad', 'Precio Unitario', 'Monto total Egreso']]

# 10. Concatenar toda la información
egresos = pd.concat([egresos_ok, egresos_nok], axis=0) \
            .drop_duplicates().drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)', 'Monto total Egreso'], keep='first')

In [24]:
egresos = egresos.merge(reporte_devoluciones, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']).drop(['Comisión %'], axis=1)

#### **Comisiones**

In [25]:
# poner comision estandar para todo caso donde no se encuentren comisiones (el proceso actual ahora lo que hace es desechar esta transacciones sin comisión) 15%

In [26]:
# 1. Separar rango de comisiones por fechas de compra

## 1. Ventas

ventas = ventas \
            .assign(**{
                'RANGO_COMISION': lambda x: np.select(
                    [
                        (x['Fecha de Compra'] < fecha_comision), 
                        (x['Fecha de Compra'] >= fecha_comision) & (x['Fecha de Compra'] < fecha_comision_2), 
                        (x['Fecha de Compra'] >= fecha_comision_2)
                    ],
                    ['A', 'B', 'C'],
                    default=''
                )
            })

## 2. Egresos

egresos = egresos \
            .assign(**{
                'RANGO_COMISION': lambda x: np.select(
                    [
                        (x['Fecha de Compra'] < fecha_comision), 
                        (x['Fecha de Compra'] >= fecha_comision) & (x['Fecha de Compra'] < fecha_comision_2), 
                        (x['Fecha de Compra'] >= fecha_comision_2)
                    ],
                    ['A', 'B', 'C'],
                    default=''
                )
            })

## 2. Procesamiento de comisiones

### 1. Ventas

#### 1. A

sin_comision_c1 = ventas.query("`RANGO_COMISION`=='A'") \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='1'").loc[:, ['RUC seller', 'Comisión %']], how='left', on=['RUC seller'])
con_comision_c1 = sin_comision_c1.query("`Comisión %`.notnull()")
sin_comision_c1 = sin_comision_c1.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop('Comisión %', axis=1)

sin_comision_c2 = sin_comision_c1 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='2'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c2 = sin_comision_c2.query("`Comisión %`.notnull()")
sin_comision_c2 = sin_comision_c2.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c3 = sin_comision_c2 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='3'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c3 = sin_comision_c3.query("`Comisión %`.notnull()")
sin_comision_c3 = sin_comision_c3.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c4 = sin_comision_c3 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='4'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c4 = sin_comision_c4.query("`Comisión %`.notnull()")
sin_comision_c4 = sin_comision_c4.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c5 = sin_comision_c4 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:3]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='5'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c5 = sin_comision_c5.query("`Comisión %`.notnull()")
sin_comision_c5 = sin_comision_c5.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c6 = sin_comision_c5 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='6'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c6 = sin_comision_c6.query("`Comisión %`.notnull()")
sin_comision_c6 = sin_comision_c6.assign(**{'Comisión %': lambda x: round(x['Comisión %'].fillna(0.15), 2)})

ventas_com_a = pd.concat([con_comision_c1, con_comision_c2, con_comision_c3, con_comision_c4, con_comision_c5, con_comision_c6, sin_comision_c6], axis=0)

#### B.

sin_comision_c1 = ventas.query("`RANGO_COMISION`=='B'") \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='1'").loc[:, ['RUC seller', 'Comisión %']], how='left', on=['RUC seller'])
con_comision_c1 = sin_comision_c1.query("`Comisión %`.notnull()")
sin_comision_c1 = sin_comision_c1.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop('Comisión %', axis=1)

sin_comision_c2 = sin_comision_c1 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='2'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c2 = sin_comision_c2.query("`Comisión %`.notnull()")
sin_comision_c2 = sin_comision_c2.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c3 = sin_comision_c2 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='3'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c3 = sin_comision_c3.query("`Comisión %`.notnull()")
sin_comision_c3 = sin_comision_c3.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c4 = sin_comision_c3 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='4'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c4 = sin_comision_c4.query("`Comisión %`.notnull()")
sin_comision_c4 = sin_comision_c4.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c5 = sin_comision_c4 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:3]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='5'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c5 = sin_comision_c5.query("`Comisión %`.notnull()")
sin_comision_c5 = sin_comision_c5.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c6 = sin_comision_c5 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='6'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c6 = sin_comision_c6.query("`Comisión %`.notnull()")
sin_comision_c6 = sin_comision_c6.assign(**{'Comisión %': lambda x: round(x['Comisión %'].fillna(0.15), 2)})

ventas_com_b = pd.concat([con_comision_c1, con_comision_c2, con_comision_c3, con_comision_c4, con_comision_c5, con_comision_c6, sin_comision_c6], axis=0)

### 3. C

sin_comision_c1 = ventas.query("`RANGO_COMISION`=='C'") \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='1'").loc[:, ['RUC seller', 'Comisión %']], how='left', on=['RUC seller'])
con_comision_c1 = sin_comision_c1.query("`Comisión %`.notnull()")
sin_comision_c1 = sin_comision_c1.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop('Comisión %', axis=1)

sin_comision_c2 = sin_comision_c1 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='2'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c2 = sin_comision_c2.query("`Comisión %`.notnull()")
sin_comision_c2 = sin_comision_c2.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c3 = sin_comision_c2 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='3'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c3 = sin_comision_c3.query("`Comisión %`.notnull()")
sin_comision_c3 = sin_comision_c3.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c4 = sin_comision_c3 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='4'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c4 = sin_comision_c4.query("`Comisión %`.notnull()")
sin_comision_c4 = sin_comision_c4.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c5 = sin_comision_c4 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:3]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='5'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c5 = sin_comision_c5.query("`Comisión %`.notnull()")
sin_comision_c5 = sin_comision_c5.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c6 = sin_comision_c5 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='6'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c6 = sin_comision_c6.query("`Comisión %`.notnull()")
sin_comision_c6 = sin_comision_c6.assign(**{'Comisión %': lambda x: round(x['Comisión %'].fillna(0.15), 2)})

ventas_com_c = pd.concat([con_comision_c1, con_comision_c2, con_comision_c3, con_comision_c4, con_comision_c5, con_comision_c6, sin_comision_c6], axis=0)

### 1. Egresos

#### 1. A

sin_comision_c1 = egresos.query("`RANGO_COMISION`=='A'") \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='1'").loc[:, ['RUC seller', 'Comisión %']], how='left', on=['RUC seller'])
con_comision_c1 = sin_comision_c1.query("`Comisión %`.notnull()")
sin_comision_c1 = sin_comision_c1.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop('Comisión %', axis=1)

sin_comision_c2 = sin_comision_c1 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='2'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c2 = sin_comision_c2.query("`Comisión %`.notnull()")
sin_comision_c2 = sin_comision_c2.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c3 = sin_comision_c2 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='3'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c3 = sin_comision_c3.query("`Comisión %`.notnull()")
sin_comision_c3 = sin_comision_c3.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c4 = sin_comision_c3 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='4'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c4 = sin_comision_c4.query("`Comisión %`.notnull()")
sin_comision_c4 = sin_comision_c4.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c5 = sin_comision_c4 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:3]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='5'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c5 = sin_comision_c5.query("`Comisión %`.notnull()")
sin_comision_c5 = sin_comision_c5.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c6 = sin_comision_c5 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='A' & `regla`=='6'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c6 = sin_comision_c6.query("`Comisión %`.notnull()")
sin_comision_c6 = sin_comision_c6.assign(**{'Comisión %': lambda x: round(x['Comisión %'].fillna(0.15), 2)})

egresos_com_a = pd.concat([con_comision_c1, con_comision_c2, con_comision_c3, con_comision_c4, con_comision_c5, con_comision_c6, sin_comision_c6], axis=0)

#### B.

sin_comision_c1 = egresos.query("`RANGO_COMISION`=='B'") \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='1'").loc[:, ['RUC seller', 'Comisión %']], how='left', on=['RUC seller'])
con_comision_c1 = sin_comision_c1.query("`Comisión %`.notnull()")
sin_comision_c1 = sin_comision_c1.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop('Comisión %', axis=1)

sin_comision_c2 = sin_comision_c1 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='2'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c2 = sin_comision_c2.query("`Comisión %`.notnull()")
sin_comision_c2 = sin_comision_c2.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c3 = sin_comision_c2 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='3'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c3 = sin_comision_c3.query("`Comisión %`.notnull()")
sin_comision_c3 = sin_comision_c3.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c4 = sin_comision_c3 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='4'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c4 = sin_comision_c4.query("`Comisión %`.notnull()")
sin_comision_c4 = sin_comision_c4.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c5 = sin_comision_c4 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:3]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='5'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c5 = sin_comision_c5.query("`Comisión %`.notnull()")
sin_comision_c5 = sin_comision_c5.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c6 = sin_comision_c5 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='B' & `regla`=='6'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c6 = sin_comision_c6.query("`Comisión %`.notnull()")
sin_comision_c6 = sin_comision_c6.assign(**{'Comisión %': lambda x: round(x['Comisión %'].fillna(0.15), 2)})

egresos_com_b = pd.concat([con_comision_c1, con_comision_c2, con_comision_c3, con_comision_c4, con_comision_c5, con_comision_c6, sin_comision_c6], axis=0)

### 3. C

sin_comision_c1 = egresos.query("`RANGO_COMISION`=='C'") \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='1'").loc[:, ['RUC seller', 'Comisión %']], how='left', on=['RUC seller'])
con_comision_c1 = sin_comision_c1.query("`Comisión %`.notnull()")
sin_comision_c1 = sin_comision_c1.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop('Comisión %', axis=1)

sin_comision_c2 = sin_comision_c1 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='2'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c2 = sin_comision_c2.query("`Comisión %`.notnull()")
sin_comision_c2 = sin_comision_c2.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c3 = sin_comision_c2 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:7]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='3'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c3 = sin_comision_c3.query("`Comisión %`.notnull()")
sin_comision_c3 = sin_comision_c3.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c4 = sin_comision_c3 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='4'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c4 = sin_comision_c4.query("`Comisión %`.notnull()")
sin_comision_c4 = sin_comision_c4.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c5 = sin_comision_c4 \
                    .assign(**{'concat': lambda x: x['RUC seller'].astype(str) + ' - ' + x['Cod.Subclase'].str[:3]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='5'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c5 = sin_comision_c5.query("`Comisión %`.notnull()")
sin_comision_c5 = sin_comision_c5.query("`Comisión %`.isnull() | `Comisión %`.isna()").drop(['Comisión %'], axis=1)

sin_comision_c6 = sin_comision_c5 \
                    .assign(**{'concat': lambda x: '0' + ' - ' + x['Cod.Subclase'].str[:5]}) \
                    .merge(comisiones.query("`RANGO_COMISION`=='C' & `regla`=='6'").loc[:, ['concat', 'Comisión %']], how='left', on=['concat']) \
                    .drop(['concat'], axis=1)
con_comision_c6 = sin_comision_c6.query("`Comisión %`.notnull()")
sin_comision_c6 = sin_comision_c6.assign(**{'Comisión %': lambda x: round(x['Comisión %'].fillna(0.15), 2)})

egresos_com_c = pd.concat([con_comision_c1, con_comision_c2, con_comision_c3, con_comision_c4, con_comision_c5, con_comision_c6, sin_comision_c6], axis=0)

#### **Estandarización de data**

In [27]:
# 1. Resumen 

## 1. Ventas

ventas = pd.concat([ventas_com_a, ventas_com_b, ventas_com_c], axis=0) \
            .merge(info_seller_q, how='left', on=['RUC seller']).rename(columns={'N ID': 'ID SELLER SVL'}) \
            .loc[:, ['Fecha de Compra', 
                 'ID SELLER SVL', 'RUC seller', 'Seller', 
                 'OC', 'Folio Relacionado', 'Producto (SKU)', 
                 'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción',
                 'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 'Local Despacho', 'Estado TRX',  
                 'Cantidad', 'Precio Unitario', 'Monto Total', 'Monto Dscto', 'Monto_Rebaja', 'Ventas', 
                 'Comisión %',
                 'Documento Tributario', 'Medio Pago', 'Descrip Medio Pago', 
                 'Cliente', 'DNI/RUC Cliente', 'Correo Cliente', 'Direccion Cliente', 
                 'Nombre Comprador', 'Apellido Comprador', 'Telefono Comprador', 
                 'Distrito', 'Provincia']] \
            .sort_values(['Fecha de Compra']) \
            .assign(**{'Cuenta': lambda x: np.where((x['Fecha de Compra'] < fecha_coe), 'FALABELLA MKP', 'FALABELLA.COM')})

## 2. Egresos

egresos = pd.concat([egresos_com_a, egresos_com_b, egresos_com_c], axis=0) \
            .drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first') \
            .merge(info_seller_q, how='left', on=['RUC seller']) \
            .rename(columns={'N ID': 'ID SELLER SVL'}) \
            .merge(template_rec_sin.loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'ID SELLER SVL', 'Serie', 'Factura']], how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)', 'ID SELLER SVL']) \
            .loc[:, ['Fecha de Compra', 
                   'ID SELLER SVL', 'RUC seller', 'Seller', 
                   'OC', 'Folio Relacionado', 'Producto (SKU)', 
                   'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción', 
                   'Tienda', 'Caja', 'Trx', 'Cod.Subclase', 'Estado Folio', 
                   'Fecha de Egreso', 
                   'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 
                   'Distrito', 'Provincia', 
                   'Despacho Directo', 'Status Recojo', #'Gestión Recojo', 'Comentario Recojo', 
                   'Estado Egreso',
                   'Encargado Egreso', 'Transporte', 'F3', 'Guia de Retorno', 
                   'Fecha de Retorno', 
                   'Motivo Devolución', 'Dirección Entrega', 'Persona Entrega', 'Link Guia', 
                   'Cantidad', 'Precio Unitario', 'Monto total Egreso', 'Comisión %', 
                   'Estado Devolución', 'Serie', 'Factura',
                   'Cliente', 'DNI/RUC Cliente',
                   'Usuario', 'CODIGO_PERFIL',
                   ]] \
            .sort_values(['Fecha de Compra']) \
            .assign(**{'Cuenta': lambda x: np.where((x['Fecha de Egreso'] < fecha_coe), 'FALABELLA MKP', 'FALABELLA.COM')})

#### **Reportería**
En este proceso se realiza el cruce de información para obtener reportes semanales.

##### **1. REPORTE DE RECAUDOS Y LIQUIDACIONES**

In [28]:
##  1. Ventas

### 1. Cruce de información

#### Consolidado de liquidaciones
cons_liq = template_liq.loc[:, ['Número Orden de Venta', 'SKU', 'Monto Total']] \
                .rename(columns={'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)'}) \
                .assign(**{'LIQUIDADO': 'SI'})

fecha_evaluacion = fecha_inicio - timedelta(14)

### 2. Procesamiento

resultado_ventas =  ventas \
                        .query("`Fecha de Compra` >= @fecha_coe") \
                        .assign(**{
                            'Categoría': lambda x: np.select(
                                [
                                    x['Cod.Subclase'].str[:7].isin(kams[kams['len']==7]['Categoría'].to_list()),
                                    x['Cod.Subclase'].str[:5].isin(kams[kams['len']==5]['Categoría'].to_list()),
                                    x['Cod.Subclase'].str[:3].isin(kams[kams['len']==3]['Categoría'].to_list())
                                ],[
                                    x['Cod.Subclase'].str[:7], 
                                    x['Cod.Subclase'].str[:5], 
                                    x['Cod.Subclase'].str[:3]
                                ],
                                default=x['Cod.Subclase']
                            ),
                            'Cod': lambda x: x['Cod.Subclase'].str[:5],
                            'F-SKU': lambda x: x['Folio Relacionado'] + ' - ' + x['Producto (SKU)'],
                            'Comisión S/': lambda x: x['Monto Total'] * x['Comisión %'],
                            'Comisión sin IGV': lambda x: (x['Monto Total'] * x['Comisión %']) / 1.18,
                            'Neto': lambda x: x['Monto Total'] - (x['Monto Total'] * x['Comisión %'])
                        }) \
                        .merge(kams.drop(['len'], axis=1), how='left', on=['Categoría']).drop_duplicates() \
                        .assign(**{
                            '# SAP': '1',
                            'STATUS': 'EXTENDIDO',
                            'Revisado': '',
                            'Estado TRX': '',
                            'Asiento venta': '',
                            'Fecha corte': '',
                            'Fecha pago': '',
                            'Pedido comisión': '',
                            'Fecha cobro': '',
                            'Detalle': '',
                            'CMR': '',
                            'Precio Real': '',
                            'PAGA': lambda x: np.select(
                                [
                                    (x['Fecha de Compra'] < fecha_evaluacion),
                                    (x['Fecha de Compra'] >= fecha_evaluacion) & (x['Estado Folio']=='Entrega Total')
                                ],[
                                    'SI',
                                    'SI'
                                ],
                                default='NO'
                            ),
                            }) \
                        .merge(cons_liq, how='left', on=['OC', 'Producto (SKU)', 'Monto Total']) \
                        .query('LIQUIDADO!="SI"') \
                        .drop(['LIQUIDADO'], axis=1) \
                        .loc[:, ['Fecha de Compra', 
                                'Cliente', 'DNI/RUC Cliente',
                                'RUC seller', 'Seller', 'KAM',
                                'OC', 'Folio Relacionado', 'Producto (SKU)',
                                'Cod', 'Linea', 'Sublinea', 'Cod.Subclase', 
                                'Estado Folio', 
                                'Descripción', 
                                'Cantidad', 'Precio Unitario',  
                                'Monto Total', 'CMR', 'Precio Real',
                                'Comisión %', 'Revisado', 'Comisión S/', 'Neto', 
                                'PAGA'
                                ]].drop_duplicates()

## 2. Egresos

resultado_egresos = egresos \
                        .query("`Fecha de Compra` >= @fecha_coe") \
                        .query("`Fecha de Egreso` >= @fecha_coe") \
                        .assign(**{
                            'Categoría': lambda x: np.select(
                                [
                                    x['Cod.Subclase'].str[:7].isin(kams[kams['len']==7]['Categoría'].to_list()),
                                    x['Cod.Subclase'].str[:5].isin(kams[kams['len']==5]['Categoría'].to_list()),
                                    x['Cod.Subclase'].str[:3].isin(kams[kams['len']==3]['Categoría'].to_list())
                                ],[
                                    x['Cod.Subclase'].str[:7], 
                                    x['Cod.Subclase'].str[:5], 
                                    x['Cod.Subclase'].str[:3]
                                ],
                                default=x['Cod.Subclase']
                            )
                        }) \
                        .merge(kams.drop(['len'], axis=1), how='left', on=['Categoría']).drop_duplicates() \
                        .assign(**{
                            '# SAP': '1',
                            'STATUS': 'EXTENDIDO'
                            }) \
                        .merge(ventas_r, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .assign(**{
                            'Comisión S/': lambda x: x['Monto Total'] * x['Comisión %'],
                            'Comisión sin IGV': lambda x: (x['Monto Total'] * x['Comisión %']) / 1.18,
                            'Neto': lambda x: x['Monto Total'] - (x['Monto Total'] * x['Comisión %']),
                            'Monto total Egreso': lambda x: np.where(x['Monto Total'] < x['Monto total Egreso']*-1, x['Monto Total']*-1, x['Monto total Egreso']),                      
                            'Cantidad': lambda x: x['Monto total Egreso'] / x['Precio Unitario'] * -1,
                            'Cond Cantidad': lambda x: x['Cantidad'] % 1 != 0
                        }) \
                        .assign(**{
                            'Monto total Egreso': lambda x: np.where((x['Cond Cantidad'] == True), x['Monto Total']*-1, x['Monto total Egreso']),
                            'Cantidad': lambda x: x['Monto total Egreso'] / x['Precio Unitario']*-1
                        }) \
                        .assign(**{'Cantidad': lambda x: x['Cantidad'].fillna(0).astype(int).abs()}) \
                        .drop(['Cond Cantidad'], axis=1) \
                        .assign(**{
                            'Cod': lambda x: x['Cod.Subclase'].str[:5],
                            'F-SKU': lambda x: x['Folio Relacionado'] + ' - ' + x['Producto (SKU)'],
                            'Revisado': '',
                            'Estado TRX': '',
                            'Asiento venta': '',
                            'Fecha corte': '',
                            'Fecha pago': '',
                            'Pedido comisión': '',
                            'Fecha cobro': '',
                            'Detalle': '',
                            'CMR': '',
                            'Precio Real': '',
                            'Precio Unitario': lambda x: x['Precio Unitario'].abs() * -1,
                            'Comisión S/': lambda x: x['Comisión S/'].abs() * -1,
                            'Comisión sin IGV': lambda x: x['Comisión sin IGV'].abs() * -1,
                            'Neto': lambda x: x['Monto Total'] - x['Comisión S/'],
                            'PAGA': lambda x: np.where((x['Encargado Egreso'].notnull()), 'SI', 'NO')
                            }) \
                        .loc[:, ['Fecha de Compra', 
                                'Cliente', 'DNI/RUC Cliente',
                                'RUC seller', 'Seller', 'KAM',
                                'OC', 'Folio Relacionado', 'Producto (SKU)',
                                'Cod', 'Linea', 'Sublinea', 'Cod.Subclase', 
                                'Estado Folio', 
                                'Descripción', 
                                'Cantidad', 'Precio Unitario',  
                                'Monto total Egreso', 'CMR', 'Precio Real',
                                'Comisión %', 'Revisado', 'Comisión S/', 'Neto', 
                                'PAGA'
                                ]].drop_duplicates() \
                        .rename(columns={'Monto total Egreso': 'Monto Total'}).drop_duplicates() \
                        .assign(**{
                                'OC': lambda x: x['OC'].str.replace('nan', '0'),
                                'Producto (SKU)': lambda x: x['Producto (SKU)'].str.replace('nan', '0'),
                                'Folio Relacionado': lambda x: x['Folio Relacionado'].str.replace('nan', '0')
                            }) \
                        .merge(cons_liq, how='left', on=['OC', 'Producto (SKU)', 'Monto Total']) \
                        .query('LIQUIDADO!="SI"') \
                        .drop(['LIQUIDADO'], axis=1)

##### 1. Consolidado de pagos

cons_pagos_fcom = template_liq \
                        .loc[:, ['Número Orden de Venta', 'Folio Relacionado', 'SKU', '% Comisión']] \
                        .drop_duplicates(['Número Orden de Venta', 'Folio Relacionado', 'SKU'], keep='last')

##### 2. Resultado de egresos
resultado_egresos_fcom = resultado_egresos \
                            .loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'Comisión %']] \
                            .drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='last') \
                            .rename(columns={'OC': 'Número Orden de Venta', 'Producto (SKU)': 'SKU', 'Comisión %': '% Comisión'})

##### 3. Unir consolidado de "comisiones"
comisiones_egresos_fcom = pd.concat([cons_pagos_fcom, resultado_egresos_fcom], axis=0).drop_duplicates(['Número Orden de Venta', 'Folio Relacionado', 'SKU'], keep='first')

ventas_resumen = ventas \
                    .loc[:, ['RUC seller', 'Seller', 'OC', 'Producto (SKU)', 'Precio Unitario', 'Monto Total', 'Cod.Subclase', 'Folio Relacionado', 'Fecha de Compra']].drop_duplicates() \
                    .drop_duplicates(['RUC seller', 'Seller', 'OC', 'Producto (SKU)', 'Precio Unitario', 'Monto Total'])

# 2. FALABELLA.COM

## 1. Ingresos

ventas_fcom = ventas \
                .query("(`Fecha de Compra` >= @fecha_inicio) & (`Fecha de Compra` <= @fecha_fin)") \
                .loc[:, ['Fecha de Compra', 'OC', 'Producto (SKU)', 'Precio Unitario', 'Monto Total', 'Estado Folio', 'RUC seller', 'Seller', 'Cod.Subclase', 'Folio Relacionado']].drop_duplicates() \
                .assign(**{
                    'Semana Recaudación': fecha_inicio.strftime("%Y%m%d") + '-' + fecha_fin.strftime("%Y%m%d"),
                    'CANTIDAD': lambda x: x['Monto Total'] / x['Precio Unitario'],
                    'Shipping': '',
                    'Descuento': ''
                    }) \
                .merge(info_seller_q, how='left', on='RUC seller') \
                .sort_values(['Fecha de Compra', 'RUC seller', 'OC', 'Producto (SKU)'], ascending=True) \
                .assign(**{'ID SKU': lambda x: x.reset_index().index + 1}) \
                .rename(columns={'Fecha de Compra': 'Fecha Transacción', 'OC': 'Numero Orden de Venta', 'Producto (SKU)': 'SKU',
                                'Precio Unitario': 'Unit Price', 'Monto Total': 'Total recaudo', 'Estado Folio': 'Tipo de Transaccion',
                                'N ID': 'ID SELLER SVL', 'RUC seller': 'RUC', 'Seller': 'NOMBRE LEGAL', 'Cod.Subclase': 'Categoría'}) \
                .assign(**{'Fecha de Compra': lambda x: x['Fecha Transacción']}) \
                .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'Numero Orden de Venta', 'ID SKU', 'SKU', 'CANTIDAD', 'Unit Price', 
                           'Shipping', 'Descuento', 'Total recaudo', 'Tipo de Transaccion', 'ID SELLER SVL', 'RUC', 'NOMBRE LEGAL', 
                           'Categoría', 'Folio Relacionado', 'Fecha de Compra']]

### 2. Egresos

egresos_fcom = egresos \
                .query("(`Fecha de Egreso` >= @fecha_inicio) & (`Fecha de Egreso` <= @fecha_fin)") \
                .loc[:, ['Fecha de Egreso', 'OC', 'Producto (SKU)',  'Monto total Egreso', 'Estado Folio', 'RUC seller', 'Seller']].drop_duplicates() \
                .assign(**{'Semana Recaudación': fecha_inicio.strftime("%Y%m%d") + '-' + fecha_fin.strftime("%Y%m%d")}) \
                .merge(ventas_resumen, how='left', on=['RUC seller', 'Seller', 'OC', 'Producto (SKU)']) \
                .assign(**{
                    'Monto total Egreso': lambda x: np.where(x['Monto total Egreso']*-1 > x['Monto Total'], x['Monto Total']*-1, x['Monto total Egreso']),
                    'Precio Unitario': lambda x: np.where(x['Precio Unitario'].isnull(), x['Monto total Egreso'], x['Precio Unitario'])
                }) \
                .assign(**{                        
                    'Monto total Egreso': lambda x: np.where(x['Precio Unitario'] > x['Monto total Egreso']*-1, x['Precio Unitario']*-1, x['Monto total Egreso']),
                    'Precio Unitario': lambda x: x['Precio Unitario'].abs() * -1,
                    'CANTIDAD': lambda x: x['Monto total Egreso'] / x['Precio Unitario'],
                    'Shipping': '',
                    'Descuento': ''
                }) \
                .query("~CANTIDAD.isna()") \
                .assign(**{
                    'CANTIDAD': lambda x: np.where(x['CANTIDAD'] % 1 < 0.5, np.floor(x['CANTIDAD']), np.ceil(x['CANTIDAD'])),
                    'Monto total Egreso': lambda x: x['CANTIDAD'] * x['Precio Unitario']
                }) \
                .drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first') \
                .merge(info_seller_q, how='left', on='RUC seller') \
                .sort_values(['Fecha de Egreso', 'RUC seller', 'OC', 'Producto (SKU)'], ascending=True) \
                .assign(**{'ID SKU': lambda x: x.reset_index().index + 1}) \
                .rename(columns={'Fecha de Egreso': 'Fecha Transacción', 'OC': 'Numero Orden de Venta', 'Producto (SKU)': 'SKU',
                                    'Precio Unitario': 'Unit Price', 'Monto total Egreso': 'Total recaudo', 'Estado Folio': 'Tipo de Transaccion',
                                    'N ID': 'ID SELLER SVL', 'RUC seller': 'RUC', 'Seller': 'NOMBRE LEGAL', 'Cod.Subclase': 'Categoría'}) \
                .assign(**{'Fecha de Egreso': lambda x: x['Fecha Transacción']}) \
                .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'Numero Orden de Venta', 'ID SKU', 'SKU', 'CANTIDAD',  'Unit Price',
                             'Shipping', 'Descuento', 'Total recaudo', 'Tipo de Transaccion', 'ID SELLER SVL', 'RUC', 'NOMBRE LEGAL', 
                             'Categoría', 'Folio Relacionado', 'Fecha de Compra', 'Fecha de Egreso']] \
                .query("`Fecha de Compra`.notnull()")

### 3. Unir tablas

hist_rec_fcom = template_rec.copy().assign(**{'aux': 'H'})

rec_fcom = pd.concat([ventas_fcom, egresos_fcom], axis=0) \
            .query("(`Folio Relacionado`.notnull()) & (`Folio Relacionado` != 'nan')") \
            .drop_duplicates(['Numero Orden de Venta', 'Folio Relacionado', 'SKU', 'Total recaudo'], keep='first') \
            .assign(**{'aux': 'N'})

rec_fcom = pd.concat([hist_rec_fcom, rec_fcom], axis=0) \
                .drop_duplicates(['Numero Orden de Venta', 'SKU', 'Total recaudo'], keep='first') \
                .assign(**{'Tipo': lambda x: np.where((x['Total recaudo']>=0), 'Ingreso', 'Egreso')}) \
                .drop_duplicates(['Numero Orden de Venta', 'SKU', 'Tipo'], keep='first') \
                .query("`aux`=='N'").drop(['aux', 'Tipo'], axis=1)

## 2. Template Liquidaciones

### 1. Pagos

pagos_fcom = resultado_ventas \
                .query("PAGA=='SI'") \
                .loc[:, ['Fecha de Compra', 'OC', 'Producto (SKU)', 'RUC seller', 'Seller', 'Estado Folio', 'Cantidad', 'Precio Unitario', 'Monto Total', 'Comisión S/', 'Neto', 'Comisión %', 'Cod.Subclase', 'Folio Relacionado']] \
                .assign(**{'Semana Recaudación': fecha_inicio.strftime("%Y%m%d") + '-' + fecha_fin.strftime("%Y%m%d")}) \
                .merge(cofi, how='left', on='Producto (SKU)') \
                .assign(**{'Tamaño': lambda x: x['Tamaño'].fillna('S')}) \
                .assign(**{
                    'Shipping': lambda x: 
                        np.where(
                        (x['Fecha de Compra'] >= fecha_cofi) & (x['Fecha de Compra'] < fecha_cofi_2), 
                        np.select(
                        [
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] < 79),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 79) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 79) & (x['Tamaño'].isin(['M', 'LO', 'L'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 79) & (x['Tamaño'].isin(['XL', 'XXL', 'O', 'EO']))
                        ],
                        [0.9, 6.9, 8.9, 19.9],
                        default=0
                        ),
                        np.where(
                        (x['Fecha de Compra'] >= fecha_cofi_2),
                        np.select(
                        [
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] < 99) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] < 99) & (x['Tamaño'].isin(['M', 'LO'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] < 99) & (x['Tamaño'].isin(['L', 'XL', 'XXL', 'O', 'EO'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['XXXS'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['XXS', 'XS'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['S', 'M', 'LO', 'L'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['XL', 'XXL'])),
                            (~x['RUC seller'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['O', 'EO']))
                        ],
                        [1, 4.9, 6.9, 5.9, 6.9, 9.9, 15.9, 19.9],
                        default=0
                        ),
                        0)
                        ),
                        'Descuento': ''
                    }) \
                .merge(info_seller_q, how='left', on='RUC seller') \
                .sort_values(['Fecha de Compra', 'RUC seller', 'OC', 'Producto (SKU)'], ascending=True) \
                .assign(**{'ID SKU': lambda x: x.reset_index().index + 1}) \
                .rename(columns={'Fecha de Compra': 'Fecha Transacción', 'OC': 'Número Orden de Venta', 'Producto (SKU)': 'SKU',
                                'N ID': 'ID SELLER SVL', 'RUC seller': 'RUC', 'Seller': 'NOMBRE LEGAL', 'Estado Folio': 'Tipo de Transaccion',
                                'Precio Unitario': 'Unit Price', 'Comisión S/': 'Importe Comisión', 'Neto': 'Importe a Pagar', 'Comisión %': '% Comisión', 'Cod.Subclase': 'Categoría'}) \
                .assign(**{'Fecha de Compra': lambda x: x['Fecha Transacción']}) \
                .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'Número Orden de Venta', 'ID SKU', 'SKU', 'ID SELLER SVL', 'RUC', 'NOMBRE LEGAL', 'Tipo de Transaccion', 'Cantidad', 'Unit Price', 'Shipping', 'Descuento', 'Monto Total', 'Importe Comisión', 'Importe a Pagar', '% Comisión', 'Categoría', 'Folio Relacionado', 'Fecha de Compra']]

### 2. Descuentos

descuentos_fcom = rec_fcom \
                    .query("`Total recaudo`<0").drop(['Shipping'], axis=1) \
                    .merge(cofi.rename(columns={'Producto (SKU)': 'SKU'}), how='left', on='SKU') \
                    .assign(**{'Tamaño': lambda x: x['Tamaño'].fillna('S')}) \
                    .assign(**{
                        'Shipping': lambda x: 
                            np.where(
                            (x['Fecha de Compra'] >= fecha_cofi) & (x['Fecha de Compra'] < fecha_cofi_2), 
                            np.select(
                            [
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] > -79),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -79) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -79) & (x['Tamaño'].isin(['M', 'LO', 'L'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -79) & (x['Tamaño'].isin(['XL', 'XXL', 'O', 'EO']))
                            ],
                            [-0.9, -6.9, -8.9, -19.9],
                            default=0
                            ),
                            np.where(
                            (x['Fecha de Compra'] >= fecha_cofi_2),
                            np.select(
                            [
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] > -99) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] > -99) & (x['Tamaño'].isin(['M', 'LO'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] > -99) & (x['Tamaño'].isin(['L', 'XL', 'XXL', 'O', 'EO'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -99) & (x['Tamaño'].isin(['XXXS'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -99) & (x['Tamaño'].isin(['XXS', 'XS'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -99) & (x['Tamaño'].isin(['S', 'M', 'LO', 'L'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -99) & (x['Tamaño'].isin(['XL', 'XXL'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Total recaudo'] <= -99) & (x['Tamaño'].isin(['O', 'EO']))
                            ],
                            [-1, -4.9, -6.9, -5.9, -6.9, -9.9, -15.9, -19.9],
                            default=0
                            ),
                            0)
                            ),
                            'Descuento': ''
                    }) \
                    .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'Numero Orden de Venta', 'ID SKU', 'SKU', 'ID SELLER SVL', 'RUC', 'NOMBRE LEGAL', 'Tipo de Transaccion', 'CANTIDAD', 'Unit Price', 'Shipping', 'Descuento', 'Total recaudo', 'Categoría', 'Folio Relacionado', 'Fecha de Compra', 'Fecha de Egreso']] \
                    .rename(columns={'Numero Orden de Venta': 'Número Orden de Venta', 'CANTIDAD': 'Cantidad', 'Total recaudo': 'Monto Total'}) \
                    .merge(comisiones_egresos_fcom, how='left', on=['Número Orden de Venta', 'Folio Relacionado', 'SKU']) \
                    .assign(**{
                        'Importe Comisión': lambda x: x['Monto Total'] * x['% Comisión'],
                        'Importe a Pagar': lambda x: x['Monto Total'] - x['Importe Comisión']
                    }) \
                    .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'Número Orden de Venta', 'ID SKU', 'SKU', 'ID SELLER SVL', 'RUC', 'NOMBRE LEGAL','Tipo de Transaccion', 'Cantidad', 'Unit Price', 'Shipping', 'Descuento', 'Monto Total', 'Importe Comisión', 'Importe a Pagar', '% Comisión', 'Categoría', 'Folio Relacionado', 'Fecha de Compra', 'Fecha de Egreso']]

### 3. Liquidación

liq_fcom = pd.concat([pagos_fcom, descuentos_fcom], axis=0) \
                .drop_duplicates(['Número Orden de Venta', 'SKU', 'Monto Total'], keep='first') \
                .assign(**{'Tipo': lambda x: np.where((x['Monto Total']>=0), 'Ingreso', 'Egreso')}) \
                .drop_duplicates(['Número Orden de Venta', 'SKU', 'Tipo'], keep='first').drop(['Tipo'], axis=1) \
                .assign(**{'Importe a Pagar': lambda x: x['Monto Total'] - x['Importe Comisión'] - x['Shipping']})

In [29]:
pd.concat([
    resultado_ventas.query("`PAGA` == 'SI'").rename(columns={'Monto Total': 'Pago MKP'}).agg({'Pago MKP': 'sum'}),
    liq_fcom.query("`Monto Total` > 0").rename(columns={'Monto Total': 'Pago F.COM'}).agg({'Pago F.COM': 'sum'}),
    rec_fcom.query("`Total recaudo` > 0").rename(columns={'Total recaudo': 'Ingreso MKP'}).agg({'Ingreso MKP': 'sum'}),
    rec_fcom.query("`Total recaudo` < 0").rename(columns={'Total recaudo': 'Egreso MKP'}).agg({'Egreso MKP': 'sum'}),
    liq_fcom.query("`Monto Total` < 0").rename(columns={'Monto Total': 'Egreso F.COM'}).agg({'Egreso F.COM': 'sum'})
], axis=0)

Pago MKP       3126935.71
Pago F.COM     3126935.71
Ingreso MKP    2118669.19
Egreso MKP     -351236.76
Egreso F.COM   -351236.76
dtype: float64

#### **2. REPORTE SEMANAL SELLERS**

In [30]:
# 1. FALABELLA.COM

## 1. Facturas liquidación

facturas_pago = pagos_coe \
                    .query("`Factura`.str.contains('PE-SVL')") \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Factura', 'Importe Pago', 'Fecha de Pago']] \
                    .assign(Documento = 'FACTURA LIQUIDACIÓN') \
                    .rename(columns={'Factura': 'Número de documento', 'Importe Pago': 'Importe'})

resumen_liq = pd.concat([template_liq, template_reg], axis=0) \
                .assign(**{
                    'Monto Total': lambda x: round(x['Monto Total'].fillna(0), 2),
                    'Importe Comisión': lambda x: round(x['Importe Comisión'].fillna(0), 2),
                    'Importe a Pagar': lambda x: round(x['Importe a Pagar'].fillna(0), 2)
                }) \
                .groupby(['ID SELLER SVL', 'Semana Recaudación', 'Tipo'], dropna=False) \
                .agg({'Monto Total': 'sum', 'Importe Comisión': 'sum', 'Importe a Pagar': 'sum'}).reset_index() \
                .drop(['Monto Total', 'Importe Comisión'], axis=1) \
                .pivot_table(index=['ID SELLER SVL', 'Semana Recaudación'], columns='Tipo', values='Importe a Pagar').reset_index() \
                .assign(**{
                    'Pago': lambda x: round(x['Pago'].fillna(0), 2),
                    'Descuento': lambda x: round(x['Descuento'].fillna(0), 2),
                    'Regularización': lambda x: round(x['Regularización'].fillna(0), 2),
                    'Liquidación': lambda x: x['Pago'] + x['Descuento'] + x['Regularización']
                }) 

facturas_pago = pd.merge(resumen_liq, facturas_pago, how='left', on=['Semana Recaudación', 'ID SELLER SVL']).assign(**{'Concepto': 'LIQUIDACIÓN'}) 

## 2. Facturas comisión

facturas_comision = facturas_coe \
                        .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Factura comisión', 'Importe factura comisión']] \
                        .assign(Documento='FACTURA COMISIÓN') \
                        .rename(columns={'Factura comisión': 'Número de documento', 'Importe factura comisión': 'Importe'})

resumen_liq_ingreso = template_liq \
                        .query("Tipo=='Pago'") \
                        .groupby(['ID SELLER SVL', 'Semana Recaudación', 'Tipo'], dropna=False) \
                        .agg({'Monto Total': 'sum', 'Importe Comisión': 'sum', 'Importe a Pagar': 'sum'}).reset_index() \
                        .drop(['Monto Total', 'Importe a Pagar'], axis=1) \
                        .pivot_table(index=['ID SELLER SVL', 'Semana Recaudación'], columns='Tipo', values='Importe Comisión').reset_index() \
                        .assign(**{
                            'Descuento': 0,
                            'Pago': lambda x: round(x['Pago'].fillna(0), 2),
                            'Liquidación': lambda x: x['Pago'] + x['Descuento']
                        })

facturas_comision = pd.merge(resumen_liq_ingreso, facturas_comision, how='left', on=['Semana Recaudación', 'ID SELLER SVL']).assign(**{'Concepto': 'COMISIÓN'}) 

## 3. Notas de crédito comisión

nc_comision = facturas_coe \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Nota de crédito comisión', 'Importe nota de crédito comisión']] \
                .assign(Documento='NOTA DE CRÉDITO COMISIÓN') \
                .rename(columns={'Nota de crédito comisión': 'Número de documento', 'Importe nota de crédito comisión': 'Importe'})

resumen_liq_egreso = template_liq \
                        .query("Tipo=='Descuento'") \
                        .groupby(['ID SELLER SVL', 'Semana Recaudación', 'Tipo'], dropna=False) \
                        .agg({'Monto Total': 'sum', 'Importe Comisión': 'sum', 'Importe a Pagar': 'sum'}).reset_index() \
                        .drop(['Monto Total', 'Importe a Pagar'], axis=1) \
                        .pivot_table(index=['ID SELLER SVL', 'Semana Recaudación'], columns='Tipo', values='Importe Comisión').reset_index() \
                        .assign(**{
                            'Descuento': lambda x: round(x['Descuento'].fillna(0), 2),
                            'Pago': 0,
                            'Liquidación': lambda x: x['Pago'] + x['Descuento']
                        })

nc_comision = pd.merge(resumen_liq_egreso, nc_comision, how='left', on=['Semana Recaudación', 'ID SELLER SVL']).assign(**{'Concepto': 'COMISIÓN'}) 

## 4. Facturas rechazo

facturas_rec = pagos_coe \
                    .query("Factura.str.contains('PE-REC')") \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Factura', 'Importe Pago', 'Fecha de Pago']] \
                    .assign(Documento='FACTURA DEVOLUCIONES') \
                    .rename(columns={'Factura': 'Número de documento', 'Importe Pago': 'Importe'})

facturas_rec = pd.concat([abonos_rec, facturas_rec], axis=0).drop_duplicates()

resumen_rec_sin = template_rec_sin \
                    .groupby(['Semana Recaudación', 'ID SELLER SVL'], dropna=False) \
                    .agg({'Importe a Pagar': 'sum'}).reset_index() \
                    .rename(columns={'Importe a Pagar': 'Devoluciones'})

facturas_rec = pd.merge(resumen_rec_sin, facturas_rec, how='left', on=['Semana Recaudación', 'ID SELLER SVL']).assign(**{'Concepto': 'DEVOLUCIONES'}) 

## 5. Facturas ajuste

facturas_ajust = pagos_coe \
                    .query("Factura.str.contains('PE-AJS')") \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Factura', 'Importe Pago', 'Fecha de Pago']] \
                    .assign(Documento='FACTURA AJUSTES') \
                    .rename(columns={'Factura': 'Número de documento', 'Importe Pago': 'Importe'})

resumen_ajust = template_ajust \
                    .groupby(['Semana Recaudación', 'ID SELLER SVL'], dropna=False) \
                    .agg({'Importe a Pagar': 'sum'}).reset_index() \
                    .rename(columns={'Importe a Pagar': 'Ajuste'})

facturas_ajust = pd.merge(resumen_ajust, facturas_ajust, how='left', on=['Semana Recaudación', 'ID SELLER SVL']).assign(**{'Concepto': 'AJUSTES'}) 

## 5. Unión

facturas_seller = pd.concat([facturas_pago, facturas_comision, nc_comision, facturas_rec, facturas_ajust], axis=0)
max_week = facturas_seller.query("Importe>0")['Semana Recaudación'].max() 

facturas_seller = facturas_seller \
                    .query("`Semana Recaudación` <= @max_week") \
                    .merge(info_seller_q, how='left', left_on=['ID SELLER SVL'], right_on=['N ID']) \
                    .merge(nombre_sellers.drop_duplicates(['RUC seller'], keep='first'), how='left', on=['RUC seller']) \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 'Concepto', 'Documento', 'Número de documento', 'Fecha de Pago', 'Importe', 'Liquidación', 'Pago', 'Descuento', 'Devoluciones', 'Ajuste', 'Regularización']] \
                    .sort_values(['Semana Recaudación', 'Liquidación']) \
                    .assign(**{
                        'n': lambda x: x.groupby(['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Concepto', 'Liquidación', 'Pago', 'Descuento', 'Regularización']).cumcount()+1
                    }) \
                    .assign(**{
                        'Liquidación': lambda x: np.where((x['n']>1) & (x['Concepto']=='LIQUIDACIÓN'), 0, x['Liquidación'].fillna(0)),
                        'Pago': lambda x: np.where((x['n']>1) & (x['Concepto']=='LIQUIDACIÓN'), 0, x['Pago'].fillna(0)),
                        'Descuento': lambda x: np.where((x['n']>1) & (x['Concepto']=='LIQUIDACIÓN'), 0, x['Descuento'].fillna(0)),
                        'Devoluciones': lambda x: x['Devoluciones'].fillna(0),
                        'Ajuste': lambda x: x['Ajuste'].fillna(0),
                        'Regularización': lambda x: np.where((x['n']>1) & (x['Concepto']=='LIQUIDACIÓN'), 0, x['Regularización'].fillna(0))
                    }) \
                    .drop(['n'], axis=1)

estado_de_liq = facturas_seller \
                    .query("Concepto=='LIQUIDACIÓN'") \
                    .sort_values(['ID SELLER SVL', 'Semana Recaudación']) \
                    .assign(**{
                        'Liquidación': lambda x: x['Liquidación'].fillna(0),
                        'Importe': lambda x: x['Importe'].fillna(0)
                    }) \
                    .assign(**{
                        'Liquidación Acumulada': lambda x: x.groupby(['ID SELLER SVL'], dropna=False).agg({'Liquidación': 'cumsum'}),
                        'Importe Acumulado': lambda x: x.groupby(['ID SELLER SVL'], dropna=False).agg({'Importe': 'cumsum'}),
                        'Diferencia': lambda x: x['Liquidación Acumulada'] - x['Importe Acumulado']
                    }) \
                    .assign(**{
                        'Comentario': lambda x: np.select(
                            [
                            (x['Número de documento'].isnull()) & (x['Diferencia']<0),
                            (x['Número de documento'].isnull()) & (x['Diferencia'] >= 0) & (x['Liquidación']>0)
                            ],
                            ['NO CORRESPONDE PAGO - SALDOS PENDIENTES POR COBRAR', 'ERROR DE PAGO - COMUNICAR AL SELLER'],
                            default='PAGO REALIZADO'
                        )
                    }) \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 'Concepto', 'Comentario']] \
                    .drop_duplicates(['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 'Concepto'], keep='first')

facturas_seller = pd.merge(facturas_seller, estado_de_liq, how='left', on=['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 'Concepto'])

### Auxiliares

sellers_id = pd.merge(nombre_sellers, info_seller_q, how='left', on=['RUC seller']).drop_duplicates(['RUC seller'], keep='first').rename(columns={'N ID': 'ID SELLER SVL'})
ventas_resumen_fechas = ventas[['OC', 'Producto (SKU)', 'Folio Relacionado', 'Fecha de Compra']].drop_duplicates(['OC', 'Producto (SKU)', 'Folio Relacionado'], keep='first')
egresos_resumen_fechas = egresos[['OC', 'Producto (SKU)', 'Folio Relacionado', 'Fecha de Egreso']].drop_duplicates(['OC', 'Producto (SKU)', 'Folio Relacionado'], keep='first')

resumen_fechas = ventas_resumen_fechas.merge(egresos_resumen_fechas, how='left', on=['OC', 'Producto (SKU)', 'Folio Relacionado'])
resumen_fechas['Fecha de Egreso'] = resumen_fechas['Fecha de Egreso'].fillna('')

## 2. RECAUDOS

rec_seller = template_rec \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL',  'Numero Orden de Venta', 'SKU', 'Tipo de Transaccion', 'CANTIDAD', 'Unit Price', 'Total recaudo', 'Fecha de Compra', 'Fecha de Egreso']] \
                .rename(columns={'Numero Orden de Venta': 'OC', 'SKU': 'Producto (SKU)', 'Tipo de Transaccion': 'Estado Folio', 'CANTIDAD': 'Cantidad', 'Unit Price': 'Precio Unitario', 'Total recaudo': 'Monto Total'}) \
                .merge(ventas_rec, how='left', on=['OC', 'Producto (SKU)']) \
                .drop(['Fecha de Compra', 'Fecha de Egreso'], axis=1) \
                .merge(resumen_fechas, how='left', on=['OC', 'Producto (SKU)', 'Folio Relacionado']) \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 
                         'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 
                         'DNI/RUC Cliente', 'Cliente', 'Distrito', 'Provincia', 
                         'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción', 'Cantidad', 'Precio Unitario', 'Monto Total', 'Fecha de Compra', 'Fecha de Egreso']] \
                .assign(**{'Tipo': lambda x: np.where((x['Monto Total']>=0), 'Ingreso', 'Egreso')})

cols = ['ID SELLER SVL', 'RUC seller', 'OC', 'Folio Relacionado']
rec_seller[cols] = rec_seller[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

rec_seller = rec_seller \
                .assign(**{'Estado Folio': lambda x: np.where((x['Tipo']=='Egreso'), 'Devolución', x['Estado Folio'])}) \
                .sort_values(['Semana Recaudación'])

## 3. LIQUIDACIONES

liq_seller = template_liq \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Número Orden de Venta', 'SKU', 'Tipo de Transaccion', 'Cantidad', 'Unit Price', '% Comisión', 'Monto Total', 'Shipping', 'Descuento', 'Importe Comisión', 'Importe a Pagar', 'Fecha de Compra', 'Fecha de Egreso']] \
                .rename(columns={'RUC': 'RUC seller', 'NOMBRE LEGAL': 'Seller', 'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)', 'Tipo de Transaccion': 'Estado Folio', 'Unit Price': 'Precio Unitario', 'Shipping': 'Cof. Logístico', 'Descuento': 'CMR', 'Importe a Pagar': 'Liquidación'}) \
                .merge(ventas_liq.drop(['RUC seller', 'Seller'], axis=1), how='left', on=['OC', 'Producto (SKU)']) \
                .drop(['Fecha de Compra', 'Fecha de Egreso'], axis=1) \
                .merge(sellers_id, how='left', on=['ID SELLER SVL']) \
                .merge(resumen_fechas, how='left', on=['OC', 'Producto (SKU)', 'Folio Relacionado']) \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 
                         'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio',
                         'DNI/RUC Cliente', 'Cliente', 'Distrito', 'Provincia', 
                         'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción', 'Cantidad', 'Precio Unitario', '% Comisión', 'Monto Total', 'Importe Comisión', 'Cof. Logístico', 'CMR', 'Liquidación', 'Fecha de Compra', 'Fecha de Egreso']]

cols = ['Monto Total', 'Importe Comisión', 'Cof. Logístico', 'CMR', 'Liquidación']
liq_seller[cols] = liq_seller[cols].apply(lambda x: x.fillna(0))

liq_seller = liq_seller \
                .assign(**{
                    'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Ingreso', 'Egreso'),
                    'Estado Folio': lambda x: np.where((x['Tipo'] == 'Ingreso'), 'Entrega Total', 'Devolución')
                })

cols = ['ID SELLER SVL', 'RUC seller', 'OC', 'Folio Relacionado']
liq_seller[cols] = liq_seller[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

liq_seller = liq_seller.sort_values(['Semana Recaudación'])

## 4. RECHAZOS Y SINIESTROS

rec_sin_seller = template_rec_sin \
                    .drop(['Tipo'], axis=1) \
                    .merge(ventas.loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'RUC seller', 'Seller', 'DNI/RUC Cliente', 'Cliente', 'Distrito', 'Provincia', 'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción']], how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .merge(resumen_fechas, how='left', on=['OC', 'Producto (SKU)', 'Folio Relacionado']) \
                    .assign(**{'Estado Folio': 'Devolución'}) \
                    .rename(columns={'Shipping': 'Cof. Logístico', 'Descuento': 'CMR', 'Unit Price': 'Precio Unitario', 'Importe a Pagar': 'Importe a Pagar', 'Tipo de Transaccion': 'Tipo'}) \
                    .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 'OC',
                            'Folio Relacionado', 'Producto (SKU)', 'Estado Folio',
                            'DNI/RUC Cliente', 'Cliente', 'Distrito', 'Provincia', 'Linea',
                            'Sublinea', 'Clase', 'Subclase', 'Descripción', 'Cantidad',
                            'Precio Unitario', '% Comisión', 'Monto Total', 'Importe Comisión',
                            'Cof. Logístico', 'CMR', 'Importe a Pagar',
                            'Fecha de Compra', 'Fecha de Egreso', 'Tipo']] \
                    .assign(**{
                                'Tipo': lambda x: np.select([
                                    (x['Tipo'].str.contains('Rechazo', na=False)), 
                                    (x['Tipo'].str.contains('Siniestro', na=False))],
                                                            ['Rechazo', 'Siniestro'],
                                                            default=x['Tipo'])
          
                                })

## 5. AJUSTES

ajust_seller = template_ajust \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'Número Orden de Venta', 'SKU', 'Tipo de Transaccion', 'Cantidad', 'Unit Price', '% Comisión', 'Monto Total', 'Shipping', 'Descuento', 'Importe Comisión', 'Importe a Pagar', 'Fecha de Compra', 'Fecha de Egreso']] \
                .rename(columns={'RUC': 'RUC seller', 'NOMBRE LEGAL': 'Seller', 'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)', 'Tipo de Transaccion': 'Estado Folio', 'Unit Price': 'Precio Unitario', 'Shipping': 'Cof. Logístico', 'Descuento': 'CMR', 'Importe a Pagar': 'Liquidación'}) \
                .merge(ventas_liq.drop(['RUC seller', 'Seller'], axis=1), how='left', on=['OC', 'Producto (SKU)']) \
                .drop(['Fecha de Compra', 'Fecha de Egreso'], axis=1) \
                .merge(sellers_id, how='left', on=['ID SELLER SVL']) \
                .merge(resumen_fechas, how='left', on=['OC', 'Producto (SKU)', 'Folio Relacionado']) \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 
                         'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio',
                         'DNI/RUC Cliente', 'Cliente', 'Distrito', 'Provincia', 
                         'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción', 'Cantidad', 'Precio Unitario', '% Comisión', 'Monto Total', 'Importe Comisión', 'Cof. Logístico', 'CMR', 'Liquidación', 'Fecha de Compra', 'Fecha de Egreso']]

cols = ['Monto Total', 'Importe Comisión', 'Cof. Logístico', 'CMR', 'Liquidación']
ajust_seller[cols] = ajust_seller[cols].apply(lambda x: x.fillna(0))

tipo_ajust = template_ajust \
                .loc[:, ['Número Orden de Venta', 'Folio Relacionado', 'SKU', 'Tipo de Transaccion', 'Importe a Pagar']] \
                .rename(columns={'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)', 'Tipo de Transaccion': 'Tipo', 'Importe a Pagar': 'Liquidación'}) \
                .assign(**{'Tipo': lambda x: x['Tipo'].astype(str).str.capitalize()}) \
                .assign(**{'Tipo': lambda x: x['Tipo'].astype(str).str.replace('confinanciamiento', 'cofinanciamiento')}) \
                .drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)', 'Liquidación'], keep='first')

ajust_seller = ajust_seller \
                .merge(tipo_ajust, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)', 'Liquidación']) \
                .assign(**{'Estado Folio': lambda x: np.where((x['Tipo'] == 'Ingreso'), 'Entrega Total', 'Devolución')})

cols = ['ID SELLER SVL', 'RUC seller', 'OC', 'Folio Relacionado']
ajust_seller[cols] = ajust_seller[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

ajust_seller = ajust_seller.sort_values(['Semana Recaudación'])

## 6. DEVOLUCIONES

devoluciones_seller = egresos.copy() \
                        .loc[:, ['ID SELLER SVL', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 'Estado Egreso', 'Fecha de Compra', 'Fecha de Egreso', 'Fecha de Retorno', 'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción', 'Encargado Egreso', 'Transporte', 'F3', 'Guia de Retorno', 'Motivo Devolución', 'Dirección Entrega', 'Persona Entrega', 'Link Guia', 'Estado Devolución', 'Serie', 'Factura']] \
                        .assign(**{'Estado Folio': 'Devolución'}) \
                        .query("(`Estado Egreso`=='ENTREGADO') | (`Estado Egreso`=='EN TIENDA') | (`Estado Egreso`=='PENDIENTE DE ENTREGA')") \
                        .query("(`Fecha de Compra`>= @fecha_coe) & (`ID SELLER SVL`.notnull()) & (`Fecha de Compra`.notnull())") \
                        .sort_values(['Fecha de Retorno'])

## 7. RESUMEN DE TRANSACCIONES

### 1. Recaudos

trans_recaudos = rec_seller \
                    .groupby(['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)','Tipo'], dropna=False) \
                    .agg({'Monto Total': 'sum'}).reset_index() \
                    .pivot_table(index=['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Monto Total').reset_index() \
                    .groupby(['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], dropna=False) \
                    .agg({'Egreso': 'sum', 'Ingreso': 'sum'}).reset_index() \
                    .assign(**{'Neto Contable': lambda x: x['Ingreso'] + x['Egreso']})

### 2. Liquidaciones

trans_liq = liq_seller \
                .groupby(['Fecha de Compra', 'Fecha de Egreso','ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo'], dropna=False) \
                .agg({'Monto Total': 'sum'}).reset_index() \
                .pivot_table(index=['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Monto Total').reset_index() \
                .groupby(['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], dropna=False) \
                .agg({'Egreso': 'sum', 'Ingreso': 'sum'}).reset_index() \
                .rename(columns={'Ingreso': 'Pago', 'Egreso': 'Descuento'}) \
                .assign(**{'Liquidación': lambda x: x['Pago'] + x['Descuento']})

### 4. Información de ventas y egresos

ventas_rtx = ventas[['Fecha de Compra', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Cod.Subclase', 'Estado Folio', 'Cantidad', 'Precio Unitario', 'Tienda', 'Caja', 'Trx', 'Local Despacho']].drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

egresos_rtx = egresos[['Fecha de Egreso', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso']].drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='last')

semanas_recaudo = rec_seller \
                    .loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo', 'Semana Recaudación']] \
                    .pivot_table(index=['OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Semana Recaudación', aggfunc=lambda x: ' '.join(x)).reset_index() \
                    .rename(columns={'Ingreso': 'Semana Recaudación Recaudo Ingreso', 'Egreso': 'Semana Recaudación Recaudo Egreso'})

semanas_liquidacion = liq_seller \
                        .loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo', 'Semana Recaudación']] \
                        .pivot_table(index=['OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Semana Recaudación', aggfunc=lambda x: ' '.join(x)).reset_index() \
                        .rename(columns={'Ingreso': 'Semana Recaudación Liquidación Ingreso', 'Egreso': 'Semana Recaudación Liquidación Egreso'})

#### Unir tablas
trans_sellers = pd.merge(trans_recaudos, trans_liq, how='left', on=['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .loc[:, ['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación']] \
                    .merge(ventas_rtx, how='left', on=['Fecha de Compra', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .merge(egresos_rtx, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .merge(semanas_recaudo, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .merge(semanas_liquidacion, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .merge(sellers_id, how='left', on=['ID SELLER SVL']) \
                    .rename(columns={'Cod.Subclase': 'Categoría'}) \
                    .loc[:, ['Fecha de Compra', 'ID SELLER SVL', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Categoría', 'Estado Folio', 'Tienda', 'Caja', 'Trx', 'Local Despacho', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'Cantidad', 'Precio Unitario', 'Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación']]

cols = ['Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación']
trans_sellers[cols] = trans_sellers[cols].apply(lambda x: x.fillna(0))

trans_sellers = trans_sellers \
                    .assign(**{
                        'CxP': lambda x: np.select(
                                            [(x['Neto Contable'] < 0), (x['Liquidación'] < 0), (x['Neto Contable'] != x['Liquidación'])],
                                            [-x['Neto Contable'], -x['Liquidación'], x['Neto Contable'] - x['Liquidación']],
                                            default=0),
                        'CxC': lambda x: x['Descuento'] - x['Egreso'],
                        'Saldo': lambda x: x['CxC'] - x['CxP']
                    })

observaciones = trans_sellers \
                    .groupby(['OC', 'Folio Relacionado', 'Producto (SKU)'], dropna=False).agg({'RUC seller': 'nunique'}).reset_index() \
                    .assign(**{'Observación': lambda x: np.where((x['RUC seller'] == 1), 'OK', 'NOK')}) \
                    .loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', 'Observación']]

comisiones_liq_ing = liq_seller \
                        .query("Tipo == 'Ingreso'") \
                        .loc[:, ['ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', '% Comisión']] \
                        .rename(columns={'% Comisión': 'Comisión %'}) \
                        .drop_duplicates()

comisiones_liq_egr = liq_seller \
                        .query("Tipo == 'Egreso'") \
                        .loc[:, ['ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', '% Comisión']] \
                        .rename(columns={'% Comisión': 'Comisión %'}) \
                        .drop_duplicates()

comisiones_liqui = pd.concat([comisiones_liq_ing, comisiones_liq_egr], axis=0) \
                    .drop_duplicates(['ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

trans_sellers = trans_sellers \
                    .merge(observaciones, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .merge(comisiones_liqui, how='left', on=['ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .loc[:, ['Fecha de Compra', 'ID SELLER SVL', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 'Observación', 'Categoría', 'Tienda', 'Caja', 'Trx', 'Local Despacho', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'Cantidad', 'Precio Unitario', 
                    'Comisión %',
                    'Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación', 'CxP', 'CxC', 'Saldo']]

## 6. LISTA DE SELLERS A COMUNICAR POR CORREO (SEMANAL) 

sellers_comunicacion = facturas_seller.query("Importe > 0") \
                            .loc[:, ['ID SELLER SVL', 'Seller']].drop_duplicates(['ID SELLER SVL'], keep='last') \
                            .assign(**{'ID SELLER SVL': lambda x: x['ID SELLER SVL'].astype(int)}) \
                            .sort_values(['ID SELLER SVL'])

# Resumen

resumen_general = facturas_seller \
                    .query("Concepto == 'LIQUIDACIÓN'") \
                    .groupby(['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller'], dropna=False) \
                    .agg({'Regularización': 'sum', 'Liquidación': 'sum', 'Importe': 'sum'}).reset_index() \
                    .rename(columns={'Regularización': 'Regularizaciones', 'Liquidación': 'Liquidación calculada', 'Importe': 'Importe abonado'})

resumen_general_pagos = liq_seller \
                            .query("Tipo == 'Ingreso'") \
                            .groupby(['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller'], dropna=False) \
                            .agg({'Monto Total': 'sum', 'Importe Comisión': 'sum', 'Cof. Logístico': 'sum', 'CMR': 'sum'}).reset_index() \
                            .rename(columns={'Monto Total': 'Ingresos por ventas', 'Importe Comisión': 'Cobro de comisión por venta', 'Cof. Logístico': 'Cobro por cofinanciamiento logístico', 'CMR': 'Cobro por oportunidades únicas CMR'})

resumen_general_descuentos = liq_seller \
                                .query("Tipo == 'Egreso'") \
                                .groupby(['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller'], dropna=False) \
                                .agg({'Monto Total': 'sum', 'Importe Comisión': 'sum', 'Cof. Logístico': 'sum', 'CMR': 'sum'}).reset_index() \
                                .rename(columns={'Monto Total': 'Descuentos por devoluciones', 'Importe Comisión': 'Reembolso de comisión por venta', 'Cof. Logístico': 'Reembolso por cofinanciamiento logístico', 'CMR': 'Reembolso por oportunidades únicas CMR'})

resumen_general = resumen_general_pagos \
                    .merge(resumen_general_descuentos, how='left', on=['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller']) \
                    .merge(resumen_general, how='left', on=['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller'])

cols = ['Ingresos por ventas', 'Cobro de comisión por venta', 'Cobro por cofinanciamiento logístico', 'Cobro por oportunidades únicas CMR', 'Descuentos por devoluciones', 'Reembolso de comisión por venta', 'Reembolso por cofinanciamiento logístico', 'Reembolso por oportunidades únicas CMR', 'Regularizaciones', 'Liquidación calculada', 'Importe abonado']
resumen_general[cols] = resumen_general[cols].apply(lambda x: x.fillna(0))

resumen_general = resumen_general \
                    .assign(**{
                        'Cobro de comisión por venta': lambda x: np.where(x['Cobro de comisión por venta']==0, 0, x['Cobro de comisión por venta'] * -1),
                        'Cobro por cofinanciamiento logístico': lambda x: np.where(x['Cobro por cofinanciamiento logístico']==0, 0, x['Cobro por cofinanciamiento logístico'] * -1),
                        'Cobro por oportunidades únicas CMR': lambda x: np.where(x['Cobro por oportunidades únicas CMR']==0, 0, x['Cobro por oportunidades únicas CMR'] * -1),
                        'Reembolso de comisión por venta': lambda x: np.where(x['Reembolso de comisión por venta']==0, 0, x['Reembolso de comisión por venta'] * -1), 
                        'Reembolso por cofinanciamiento logístico': lambda x: np.where(x['Reembolso por cofinanciamiento logístico']==0, 0, x['Reembolso por cofinanciamiento logístico'] * -1), 
                        'Reembolso por oportunidades únicas CMR': lambda x: np.where(x['Reembolso por oportunidades únicas CMR']==0, 0, x['Reembolso por oportunidades únicas CMR'] * -1)
                        }) \
                    .melt(
                        id_vars=['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller'],
                        value_vars=['Ingresos por ventas', 'Cobro de comisión por venta', 'Cobro por cofinanciamiento logístico', 'Cobro por oportunidades únicas CMR', 'Descuentos por devoluciones', 'Reembolso de comisión por venta', 'Reembolso por cofinanciamiento logístico', 'Reembolso por oportunidades únicas CMR', 'Regularizaciones', 'Liquidación calculada', 'Importe abonado'],
                        var_name = 'Concepto',
                        value_name = 'Monto'
                        ) \
                    .sort_values(['Semana Recaudación', 'ID SELLER SVL'], ascending=True) \
                    .rename({'variable': 'Concepto', 'value': 'Monto'})

#### **3. REGULARIZACIONES DESCUENTOS INJUSTIFICADOS**

In [31]:
## Seleccionar trans con CxP pendientes, con fecha de compra luego del 11/04/22 y con recaudo ingreso > 0 (esto evita )

max_fecha_template = template_liq['Fecha Transacción'].max()
max_semana_recaudacion = liq_fcom['Semana Recaudación'].max()

comisiones_liq = liq_seller.query("Tipo=='Egreso'").loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', '% Comisión']].drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

reg_descuentos = trans_sellers[(trans_sellers['Fecha de Compra'] >= fecha_coe) & (trans_sellers['Fecha de Compra'] <= max_fecha_template) & 
                                (trans_sellers['Ingreso'] > 0) & (trans_sellers['Egreso'] < 0) & 
                                (trans_sellers['Pago'] == 0) & (trans_sellers['Descuento'] < 0) &
                                (trans_sellers['CxP'] > 0)] \
                    .assign(**{
                        'Semana Recaudación': max_semana_recaudacion,
                        'ID SKU': lambda x: x.reset_index().index + 1,
                        'Fecha Transacción': lambda x: x['Fecha de Compra']
                    }) \
                    .assign(**{
                        'Shipping': 0,
                        'Descuento': '',
                    }) \
                    .merge(comisiones_liq, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                    .assign(**{
                            'Importe Comisión': lambda x: x['Ingreso'] * x['% Comisión'],
                            'Importe a Pagar': lambda x: x['Ingreso'] - x['Importe Comisión']
                        }) \
                    .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'OC', 'ID SKU', 'Producto (SKU)', 'ID SELLER SVL', 'RUC seller', 'Seller', 'Estado Folio', 'Cantidad', 'Precio Unitario', 'Shipping', 'Descuento', 'Ingreso', 'Importe Comisión', 'Importe a Pagar', '% Comisión', 'Categoría', 'Folio Relacionado', 'Fecha de Compra', 'Fecha de Egreso']] \
                    .rename(columns={'OC': 'Número Orden de Venta', 'Producto (SKU)': 'SKU', 'RUC seller': 'RUC', 'Seller': 'NOMBRE LEGAL', 'Estado Folio': 'Tipo de Transaccion', 'Precio Unitario': 'Unit Price', 'Ingreso': 'Monto Total'})

## Verificación de registros (evitar egreso doble) 

hist_liq_fcom = template_liq.copy().assign(**{'aux': 'H'})

liq_reg_fcom = pd.concat([liq_fcom, reg_descuentos], axis=0) \
                    .drop_duplicates(['Número Orden de Venta', 'Folio Relacionado', 'SKU', 'Monto Total'], keep='first') \
                    .assign(**{'aux': 'N'})

liq_reg_fcom = pd.concat([hist_liq_fcom, liq_reg_fcom], axis=0) \
                    .drop_duplicates(['Número Orden de Venta', 'SKU', 'Monto Total'], keep='first') \
                    .assign(**{
                        'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Ingreso', 'Egreso'),
                        'Tipo de Transaccion': lambda x: np.where((x['Tipo'] == 'Ingreso'), 'Entrega Total', 'Devolución')
                    }) \
                    .drop_duplicates(['Número Orden de Venta', 'SKU', 'Tipo'], keep='first') \
                    .query("`aux`=='N'").drop(['Tipo', 'aux'], axis=1)

In [32]:
# 1. Históricos

## 1. Recaudos

total_recaudos = pd.concat([template_rec, rec_fcom]) \
                    .rename(columns={'Numero Orden de Venta': 'OC', 'SKU': 'Producto (SKU)', 'Total recaudo': 'Monto Total'}) \
                    .assign(**{'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Ingreso', 'Egreso')})

total_trans_recaudos = total_recaudos \
                        .groupby(['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo'], dropna=False) \
                        .agg({'Monto Total': 'sum'}).reset_index() \
                        .pivot_table(index=['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Monto Total').reset_index() \
                        .groupby(['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], dropna=False) \
                        .agg({'Egreso': 'sum', 'Ingreso': 'sum'}).reset_index() \
                        .rename(columns={'Ingreso': 'Ingreso', 'Egreso': 'Egreso'}) \
                        .assign(**{'Neto Contable': lambda x: x['Ingreso'] + x['Egreso']})

## 2. Liquidaciones

total_liq = pd.concat([template_liq, liq_reg_fcom]) \
                .rename(columns={'Número Orden de Venta': 'OC', 'SKU': 'Producto (SKU)'}) \
                .assign(**{
                    'Fecha Transacción': lambda x: pd.to_datetime(x['Fecha Transacción'], infer_datetime_format=True, errors='coerce').dt.date,
                    'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Ingreso', 'Egreso')
                })

total_trans_liq = total_liq \
                    .groupby(['Fecha de Compra', 'Fecha de Egreso','ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo'], dropna=False) \
                    .agg({'Monto Total': 'sum'}).reset_index() \
                    .pivot_table(index=['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Monto Total').reset_index() \
                    .groupby(['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)'], dropna=False) \
                    .agg({'Egreso': 'sum', 'Ingreso': 'sum'}).reset_index() \
                    .rename(columns={'Ingreso': 'Pago', 'Egreso': 'Descuento'}) \
                    .assign(**{'Liquidación': lambda x: x['Pago'] + x['Descuento']})

### 3. Resumen

semanas_recaudo = total_recaudos[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo', 'Semana Recaudación']] \
                    .pivot_table(index=['OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Semana Recaudación', aggfunc=lambda x: ' '.join(x)).reset_index() \
                    .rename(columns={'Ingreso': 'Semana Recaudación Recaudo Ingreso', 'Egreso': 'Semana Recaudación Recaudo Egreso'})

semanas_liquidacion = total_liq[['OC', 'Folio Relacionado', 'Producto (SKU)', 'Tipo', 'Semana Recaudación']] \
                        .pivot_table(index=['OC', 'Folio Relacionado', 'Producto (SKU)'], columns='Tipo', values='Semana Recaudación', aggfunc=lambda x: ' '.join(x)).reset_index() \
                        .rename(columns={'Ingreso': 'Semana Recaudación Liquidación Ingreso', 'Egreso': 'Semana Recaudación Liquidación Egreso'})

total_trans_sellers = total_trans_recaudos \
                        .merge(total_trans_liq, how='left', on=['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .loc[:, ['Fecha de Compra', 'ID SELLER SVL', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación']] \
                        .merge(ventas_rtx, how='left', on=['Fecha de Compra', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .merge(egresos_rtx, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .merge(semanas_recaudo, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .merge(semanas_liquidacion, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .merge(sellers_id, how='left', on=['ID SELLER SVL']) \
                        .rename(columns={'Cod.Subclase': 'Categoría'}) \
                        .loc[:, ['Fecha de Compra', 'ID SELLER SVL', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Categoría', 'Estado Folio', 'Tienda', 'Caja', 'Trx', 'Local Despacho', 'Fecha de Egreso', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'Cantidad', 'Precio Unitario', 'Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación']]

cols = ['Ingreso', 'Egreso', 'Neto Contable', 'Pago', 'Descuento', 'Liquidación']
total_trans_sellers[cols] = total_trans_sellers[cols].apply(lambda x: x.fillna(0))

### 8. Cálculo de saldos

total_trans_sellers = total_trans_sellers \
                        .assign(**{
                            'CxP': lambda x: np.select(
                                                [(x['Neto Contable'] < 0), (x['Liquidación'] < 0), (x['Neto Contable'] != x['Liquidación'])],
                                                [-x['Neto Contable'], -x['Liquidación'], x['Neto Contable'] - x['Liquidación']],
                                                default=0),
                            'CxC': lambda x: x['Descuento'] - x['Egreso'],
                            'Saldo': lambda x: x['CxC'] - x['CxP']
                        })

max_fecha_template = total_liq['Fecha Transacción'].max()
max_semana_recaudacion = liq_fcom['Semana Recaudación'].max()

comisiones_liq = total_liq.query("Tipo=='Egreso'").loc[:, ['OC', 'Folio Relacionado', 'Producto (SKU)', '% Comisión']].drop_duplicates(['OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

reg_descuentos_cw = total_trans_sellers[(total_trans_sellers['Fecha de Compra'] >= fecha_coe) & 
                                        (total_trans_sellers['Fecha de Compra'] <= max_fecha_template) & 
                                        (total_trans_sellers['Ingreso'] > 0) & (total_trans_sellers['Egreso'] < 0) & 
                                        (total_trans_sellers['Pago'] == 0) & (total_trans_sellers['Descuento'] < 0) &
                                        (total_trans_sellers['CxP'] > 0)] \
                        .assign(**{
                            'Semana Recaudación': max_semana_recaudacion,
                            'ID SKU': lambda x: x.reset_index().index + 1,
                            'Fecha Transacción': lambda x: x['Fecha de Compra']
                        }) \
                        .assign(**{
                            'Shipping': 0,
                            'Descuento': '',
                        }) \
                        .merge(comisiones_liq, how='left', on=['OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .assign(**{
                            'Importe Comisión': lambda x: x['Ingreso'] * x['% Comisión'],
                            'Importe a Pagar': lambda x: x['Ingreso'] - x['Importe Comisión']
                        }) \
                        .loc[:, ['Semana Recaudación', 'Fecha Transacción', 'OC', 'ID SKU', 'Producto (SKU)', 'ID SELLER SVL', 'RUC seller', 'Seller', 'Estado Folio', 'Cantidad', 'Precio Unitario', 'Shipping', 'Descuento', 'Ingreso', 'Importe Comisión', 'Importe a Pagar', '% Comisión', 'Categoría', 'Folio Relacionado', 'Fecha de Compra', 'Fecha de Egreso']] \
                        .rename(columns={'OC': 'Número Orden de Venta', 'Producto (SKU)': 'SKU', 'RUC seller': 'RUC', 'Seller': 'NOMBRE LEGAL', 'Estado Folio': 'Tipo de Transaccion', 'Precio Unitario': 'Unit Price', 'Ingreso': 'Monto Total'})


## Agregar data a liq_fcom

total_hist_liq_fcom = template_liq.copy().assign(**{'aux': 'H'})

total_liq_reg_fcom = pd.concat([liq_reg_fcom, reg_descuentos_cw], axis=0) \
                        .drop_duplicates(['Número Orden de Venta', 'Folio Relacionado', 'SKU', 'Monto Total'], keep='first') \
                        .assign(**{'aux': 'N'})

total_liq_reg_fcom = pd.concat([total_hist_liq_fcom, total_liq_reg_fcom], axis=0) \
                        .drop_duplicates(['Número Orden de Venta', 'SKU', 'Monto Total'], keep='first') \
                        .assign(**{
                            'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Ingreso', 'Egreso'),
                            'Tipo de Transaccion': lambda x: np.where((x['Tipo'] == 'Ingreso'), 'Entrega Total', 'Devolución')
                        }) \
                        .drop_duplicates(['Número Orden de Venta', 'SKU', 'Tipo'], keep='first') \
                        .query("`aux`=='N'").drop(['Tipo', 'aux'], axis=1)

total_pagos_reg_fcom = total_liq_reg_fcom \
                        .query("`Monto Total`>0") \
                        .merge(cofi.rename(columns={'Producto (SKU)': 'SKU'}), how='left', on='SKU') \
                        .assign(**{'Tamaño': lambda x: x['Tamaño'].fillna('S')}) \
                        .assign(**{
                            'Shipping': lambda x: 
                                np.where(
                                (x['Fecha de Compra'] >= fecha_cofi) & (x['Fecha de Compra'] < fecha_cofi_2), 
                                np.select(
                                [
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] < 79),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 79) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 79) & (x['Tamaño'].isin(['M', 'LO', 'L'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 79) & (x['Tamaño'].isin(['XL', 'XXL', 'O', 'EO']))
                                ],
                                [0.9, 6.9, 8.9, 19.9],
                                default=0
                                ),
                                np.where(
                                (x['Fecha de Compra'] >= fecha_cofi_2),
                                np.select(
                                [
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] < 99) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] < 99) & (x['Tamaño'].isin(['M', 'LO'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] < 99) & (x['Tamaño'].isin(['L', 'XL', 'XXL', 'O', 'EO'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['XXXS'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['XXS', 'XS'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['S', 'M', 'LO', 'L'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['XL', 'XXL'])),
                                    (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] >= 99) & (x['Tamaño'].isin(['O', 'EO']))
                                ],
                                [1, 4.9, 6.9, 5.9, 6.9, 9.9, 15.9, 19.9],
                                default=0
                                ),
                                0)
                                ),
                                'Descuento': ''
                            }) \
                        .drop(['Tamaño'], axis=1)

total_desc_reg_fcom = total_liq_reg_fcom \
                    .query("`Monto Total`<0") \
                    .merge(cofi.rename(columns={'Producto (SKU)': 'SKU'}), how='left', on='SKU') \
                    .assign(**{'Tamaño': lambda x: x['Tamaño'].fillna('S')}) \
                    .assign(**{
                        'Shipping': lambda x: 
                            np.where(
                            (x['Fecha de Compra'] >= fecha_cofi) & (x['Fecha de Compra'] < fecha_cofi_2), 
                            np.select(
                            [
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] > -79),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -79) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -79) & (x['Tamaño'].isin(['M', 'LO', 'L'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -79) & (x['Tamaño'].isin(['XL', 'XXL', 'O', 'EO']))
                            ],
                            [-0.9, -6.9, -8.9, -19.9],
                            default=0
                            ),
                            np.where(
                            (x['Fecha de Compra'] >= fecha_cofi_2),
                            np.select(
                            [
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] > -99) & (x['Tamaño'].isin(['XXXS', 'XXS', 'XS', 'S'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] > -99) & (x['Tamaño'].isin(['M', 'LO'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] > -99) & (x['Tamaño'].isin(['L', 'XL', 'XXL', 'O', 'EO'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -99) & (x['Tamaño'].isin(['XXXS'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -99) & (x['Tamaño'].isin(['XXS', 'XS'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -99) & (x['Tamaño'].isin(['S', 'M', 'LO', 'L'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -99) & (x['Tamaño'].isin(['XL', 'XXL'])),
                                (~x['RUC'].isin(sellers_ddp)) & (x['Monto Total'] <= -99) & (x['Tamaño'].isin(['O', 'EO']))
                            ],
                            [-1, -4.9, -6.9, -5.9, -6.9, -9.9, -15.9, -19.9],
                            default=0
                            ),
                            0)
                            ),
                            'Descuento': ''
                    }) \
                    .drop(['Tamaño'], axis=1)

total_liq_reg_fcom = pd.concat([total_pagos_reg_fcom, total_desc_reg_fcom], axis=0) \
                        .assign(**{
                            'Importe a Pagar': lambda x: x['Monto Total'] - x['Importe Comisión'] - x['Shipping']
                        }) \
                        .query("`Fecha de Compra` <= @fecha_fin") \
                        .drop_duplicates(['Número Orden de Venta', 'SKU', 'Monto Total'], keep='first')

In [33]:
# Formatos

liq_seller = liq_seller \
                .assign(**{
                    '% Comisión': lambda x: x['% Comisión'].astype(float).map(lambda x: '{:.2%}'.format(x)),
                    'Importe Comisión': lambda x: round(x['Importe Comisión'].fillna(0), 2),
                    'Cof. Logístico': lambda x: round(x['Cof. Logístico'].fillna(0), 2),
                    'CMR': lambda x: round(x['CMR'].fillna(0), 2),
                    'Liquidación': lambda x: round(x['Liquidación'].fillna(0), 2)
                })

rec_sin_seller = rec_sin_seller \
                    .assign(**{
                        #'% Comisión': lambda x: x['% Comisión'].astype(float).map(lambda x: '{:.2%}'.format(x)),
                        'Importe Comisión': lambda x: round(x['Importe Comisión'].fillna(0), 2),
                        'Cof. Logístico': lambda x: round(x['Cof. Logístico'].fillna(0), 2),
                        'CMR': lambda x: round(x['CMR'].fillna(0), 2),
                        'Importe a Pagar': lambda x: round(x['Importe a Pagar'].fillna(0), 2)
                    })

ajust_seller = ajust_seller \
                    .assign(**{
                        #'% Comisión': lambda x: x['% Comisión'].astype(float).map(lambda x: '{:.2%}'.format(x)),
                        'Importe Comisión': lambda x: round(x['Importe Comisión'].fillna(0), 2),
                        'Cof. Logístico': lambda x: round(x['Cof. Logístico'].fillna(0), 2),
                        'CMR': lambda x: round(x['CMR'].fillna(0), 2),
                        'Liquidación': lambda x: round(x['Liquidación'].fillna(0), 2)
                    })

#### **4. CRUCE SRX vs RECAUDOS F.COM**

In [34]:
rec_fcom = rec_fcom.query("`Fecha de Compra` >= @fecha_coe")
total_liq_reg_fcom = total_liq_reg_fcom.query("`Fecha de Compra` >= @fecha_coe")

In [35]:
# 1. Info Original SRX

## 1. Ventas

ingresos_srx = ventas_.rename(columns=lambda x: x.strip()).copy() \
                    .query("Item.notnull() & Seller.notnull()") \
                    .assign(**{
                        'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Estado Folio': lambda x: x['Estado Folio'].str.strip()
                    })

cols = ['Seller', 'RUC seller', 'Folio Relacionado', 'Producto (SKU)', 'OC', 'Tienda', 'Caja', 'Trx', 'Local Despacho']
ingresos_srx[cols] = ingresos_srx[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

ingresos_srx = ingresos_srx \
                    .assign(**{
                        'Monto Total': lambda x: pd.to_numeric(x['Monto Total'], errors='coerce').replace(np.nan, 0),
                        'Monto Dscto': lambda x: pd.to_numeric(x['Monto Dscto'], errors='coerce').replace(np.nan, 0),
                        'Monto_Rebaja': lambda x: pd.to_numeric(x['Monto_Rebaja'], errors='coerce').replace(np.nan, 0)
                    }) \
                    .assign(**{
                        'Ventas': lambda x: x['Monto Total'] - x['Monto Dscto'] - x['Monto_Rebaja'],
                        'Cantidad': lambda x: x['Monto Total'] / x['Precio Unitario']
                    }) \
                    .query("`Fecha de Compra` >= @fecha_coe") \
                    .loc[:, ['Item', 
                         'Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio',
                         'Tienda', 'Caja', 'Trx',
                         'Monto Total', 
                         'Monto Dscto', 'Monto_Rebaja', 'Ventas',
                         'Cantidad', 'Precio Unitario'
                         ]] \
                     .assign(**{'Tipo': 'Ingreso'}) 

## 2. Egresos

egresos_srx = egresos_.rename(columns=lambda x: x.strip()).copy() \
                    .rename(columns={'SKU': 'Producto (SKU)', 'Referencia OC': 'Folio Relacionado', 
                                    'Código de Local': 'Tienda Egreso', 'Caja de Egreso': 'Caja Egreso', 'Numero de transaccion Egreso': 'Trx Egreso',
                                    'Local Trx Orig': 'Tienda', 'Caja Trx Orig': 'Caja', 'Sec Trx Original': 'Trx'}) \
                    .assign(**{
                        'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], format='%d/%m/%Y', errors='coerce').dt.date,
                        'Fecha Trx Orig': lambda x: pd.to_datetime(x['Fecha Trx Orig'].str.title(), format='%d-%b-%y', errors='coerce').dt.date,
                        'RUC seller': lambda x: x['Seller'].str.extract('(^[0-9]{1,11})'),
                        'Estado Folio': lambda x: x['Estado Folio'].str.strip()
                    })

cols = ['Seller', 'RUC seller', 'Folio Relacionado', 'Producto (SKU)', 'OC', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso', 'EAN Egreso', 'SKU Egreso', 'Tienda', 'Caja', 'Trx']
egresos_srx[cols] = egresos_srx[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

egresos_srx = egresos_srx \
                    .assign(**{
                        'Seller': lambda x: x['Seller'].str.extract('(?<=^[0-9]{10}-)(.*)'),
                        'Monto total Egreso': lambda x: x['Monto total Egreso'].replace(np.nan, 0)
                    }) \
                    .query("`Fecha de Egreso` >= @fecha_coe") \
                    .loc[:, ['Item', 
                           'Fecha de Egreso', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 
                           'Tienda Egreso', 'Caja Egreso', 'Trx Egreso',
                           'Monto total Egreso'
                           ]] \
                     .assign(**{'Tipo': 'Egreso'}) \
                     .rename(columns={'Tienda Egreso': 'Tienda', 'Caja Egreso': 'Caja', 'Trx Egreso': 'Trx', 'Monto total Egreso': 'Monto Total'})

# 2. Recaudos

## 0. Recaudos 

rec_srx_fcom = pd.concat([template_rec, rec_fcom]) \
                .rename(columns={'Numero Orden de Venta': 'OC', 'SKU': 'Producto (SKU)', 'Tipo de Transaccion': 'Estado Folio', 'CANTIDAD': 'Cantidad', 'Unit Price': 'Precio Unitario', 'Total recaudo': 'Monto Total'}) \
                .drop(['Folio Relacionado'], axis=1) \
                .merge(ventas.loc[:, ['OC', 'Producto (SKU)', 'Folio Relacionado']].drop_duplicates(['OC', 'Producto (SKU)'], keep='first'), 
                        how='left', on=['OC', 'Producto (SKU)']) \
                .merge(ventas_rec, how='left', on=['OC', 'Producto (SKU)', 'Folio Relacionado']) \
                .loc[:, ['Semana Recaudación', 'ID SELLER SVL', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 
                'Linea', 'Sublinea', 'Clase', 'Subclase', 'Descripción', 'Cantidad', 'Precio Unitario', 'Monto Total', 'Fecha de Compra', 'Fecha de Egreso']] \
                .assign(**{
                    'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                    'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], infer_datetime_format=True, errors='coerce').dt.date,
                    'Tipo': lambda x: np.where((x['Monto Total'] >= 0), 'Ingreso', 'Egreso')
                })

cols = ['ID SELLER SVL', 'RUC seller', 'OC', 'Folio Relacionado']
rec_srx_fcom[cols] = rec_srx_fcom[cols].apply(lambda x: x.astype(str).str.replace('(\.0$)', ''))

## 1. Copiar información

ingresos_srx_trx = ventas \
                        .loc[:, ['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda', 'Caja', 'Trx']] \
                        .drop_duplicates(['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

egresos_srx_trx = egresos \
                        .loc[:, ['Fecha de Compra', 'Fecha de Egreso', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Tienda Egreso', 'Caja Egreso', 'Trx Egreso']] \
                        .rename(columns={'Tienda Egreso': 'Tienda', 'Caja Egreso': 'Caja', 'Trx Egreso': 'Trx'}) \
                        .drop_duplicates(['Fecha de Compra', 'Fecha de Egreso', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)'], keep='first')

ingresos_srx_fcom = rec_srx_fcom \
                        .query("Tipo == 'Ingreso'") \
                        .assign(**{'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date}) \
                        .merge(ingresos_srx_trx, how='left', on=['Fecha de Compra', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .assign(**{
                            'Considerar': 'SI',
                            'Fecha Transacción': lambda x: x['Fecha de Compra'] 
                        })

egresos_srx_fcom = rec_srx_fcom \
                        .query("Tipo == 'Egreso'") \
                        .assign(**{'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], infer_datetime_format=True, errors='coerce').dt.date}) \
                        .merge(egresos_srx_trx, how='left', on=['Fecha de Compra', 'Fecha de Egreso', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)']) \
                        .assign(**{
                            'Considerar': 'SI',
                            'Fecha Transacción': lambda x: x['Fecha de Egreso'] 
                        })

ingresos_srx_trx = ingresos_srx_fcom.loc[:, ['Tienda', 'Caja', 'Trx']].drop_duplicates().assign(**{'Considerar': 'SI'})
egresos_srx_trx = egresos_srx_fcom.loc[:, ['Tienda', 'Caja', 'Trx']].drop_duplicates().assign(**{'Considerar': 'SI'})

# 3. Cruce con información original

ingresos_srx = ingresos_srx \
                    .merge(ingresos_srx_trx, how='left', on=['Tienda', 'Caja', 'Trx']) \
                    .assign(**{
                        'Considerar': lambda x: x['Considerar'].fillna('NO'),
                        'Fecha de Compra': lambda x: pd.to_datetime(x['Fecha de Compra'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha Transacción': lambda x: x['Fecha de Compra'] 
                    })

egresos_srx = egresos_srx \
                    .merge(egresos_srx_trx, how='left', on=['Tienda', 'Caja', 'Trx']) \
                    .assign(**{
                        'Considerar': lambda x: x['Considerar'].fillna('NO'),
                        'Fecha de Egreso': lambda x: pd.to_datetime(x['Fecha de Egreso'], infer_datetime_format=True, errors='coerce').dt.date,
                        'Fecha Transacción': lambda x: x['Fecha de Egreso'] 
                    })

# 5. Recaudos
rec_srx = pd.concat([ingresos_srx, egresos_srx], axis=0) \
                .assign(**{'Origen': 'SRX'}) \
                .loc[:, ['Item', 'Fecha Transacción', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 'Tienda', 'Caja', 'Trx', 'Cantidad', 'Precio Unitario', 'Monto Total', 'Monto Dscto', 'Monto_Rebaja', 'Ventas',  'Fecha de Compra', 'Fecha de Egreso', 'Origen',   'Tipo', 'Considerar']] \
                .query("(`Fecha Transacción` >= @fecha_inicio) & (`Fecha Transacción` <= @fecha_fin)")

rec_srx_fcom = pd.concat([ingresos_srx_fcom, egresos_srx_fcom], axis=0) \
                    .assign(**{'Origen': 'F.COM'}) \
                    .loc[:, ['Fecha Transacción', 'RUC seller', 'Seller', 'OC', 'Folio Relacionado', 'Producto (SKU)', 'Estado Folio', 'Tienda', 'Caja', 'Trx', 'Cantidad', 'Precio Unitario', 'Monto Total',  'Fecha de Compra', 'Fecha de Egreso', 'Origen', 'Tipo', 'Considerar']] \
                    .query("(`Fecha Transacción` >= @fecha_inicio) & (`Fecha Transacción` <= @fecha_fin)")

# 9. Montos correctos

montos_fcom = rec_srx_fcom.query("Tipo =='Egreso'") \
                .loc[:, ['Tienda', 'Caja', 'Trx', 'Tipo', 'Monto Total']] \
                .drop_duplicates(['Tienda', 'Caja', 'Trx', 'Tipo'], keep='first') \
                .rename(columns={'Monto Total': 'Monto Correcto'})

rec_srx = pd.merge(rec_srx, montos_fcom, how='left', on=['Tienda', 'Caja', 'Trx', 'Tipo'])

# 11. Unir tablas

validacion = pd.concat([rec_srx, rec_srx_fcom], axis=0) \
                .assign(**{
                    'Considerar': lambda x: np.where(((x['Monto Correcto']).isnull() & (x['Origen']=='SRX') & (x['Tipo']=='Egreso')), 'NO', x['Considerar']),
                    'Monto Correcto': lambda x: np.where((x['Monto Correcto'].isnull()), x['Monto Total'], x['Monto Correcto'])
                }) \
                .assign(**{
                    'Monto Correcto': lambda x: x['Monto Correcto'].astype(float),
                    'Monto Total': lambda x: x['Monto Total'].astype(float),
                    'Diferencia': lambda x: x['Monto Correcto'] - x['Monto Total']
                })

##### **Resumen**

In [36]:
pd.concat([
    resultado_ventas.query("`PAGA` == 'SI'").rename(columns={'Monto Total': 'Pago MKP'}).agg({'Pago MKP': 'sum'}),
    liq_fcom.query("`Monto Total` > 0").rename(columns={'Monto Total': 'Pago F.COM'}).agg({'Pago F.COM': 'sum'}),
    total_liq_reg_fcom.query("`Monto Total` > 0").rename(columns={'Monto Total': 'Pago F.COM'}).agg({'Pago F.COM': 'sum'}),
    rec_fcom.query("`Total recaudo` > 0").rename(columns={'Total recaudo': 'Ingreso MKP'}).agg({'Ingreso MKP': 'sum'}),
    rec_fcom.query("`Total recaudo` < 0").rename(columns={'Total recaudo': 'Egreso MKP'}).agg({'Egreso MKP': 'sum'}),
    liq_fcom.query("`Monto Total` < 0").rename(columns={'Monto Total': 'Egreso F.COM'}).agg({'Egreso F.COM': 'sum'}),
    total_liq_reg_fcom.query("`Monto Total` < 0").rename(columns={'Monto Total': 'Egreso F.COM'}).agg({'Egreso F.COM': 'sum'})
], axis=0)

Pago MKP       3126935.71
Pago F.COM     3126935.71
Pago F.COM     2187415.68
Ingreso MKP    2118669.19
Egreso MKP     -351236.76
Egreso F.COM   -351236.76
Egreso F.COM   -351236.76
dtype: float64

#### **5. TRANSACCIONES**

In [37]:
rep_transacciones = trans_sellers \
                        .merge(reporte_devoluciones.loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Encargado Egreso']], 
                                how='left', on=['Folio Relacionado', 'Producto (SKU)']) \
                        .assign(**{
                            'CxP': lambda x: round(x['CxP'].fillna(0), 2),
                            'CxC': lambda x: round(x['CxC'].fillna(0), 2),
                            'Saldo': lambda x: round(x['Saldo'].fillna(0), 2)
                        })


In [38]:
rep_transacciones.agg({'CxP': 'sum', 'CxC': 'sum', 'Saldo': 'sum'})

CxP      2916785.24
CxC         8068.57
Saldo   -2908716.67
dtype: float64

#### **Exportar info**

##### **1. Reportes semanales para los KAMs**

In [39]:
if reporte_semanal_sellers!='yes':

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','')

    ## Listado de tablas a exportar
    lista = [resultado_ventas, resultado_egresos]
    hojas = ['BASE FACTURA', 'BASE ABONO']

    ## Nombre del archivo Excel
    writer = pd.ExcelWriter(os.path.join(final_path, ('Reporte de pagos y descuentos.xlsx')), engine='xlsxwriter')
            
    ## Exportar Excel
    for j, df in enumerate(lista):
            ## Excel
            df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
            ## Define workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[hojas[j]]
            ## Header format
            header_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top', 'fg_color': '#D7E4BC'})
            ## Write headers with the defined format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            ## Format Workbook
            workbook.formats[0].set_font_size(10) # Tamaño 10 letra
            ## Format Worksheet
            worksheet.set_zoom(70) # 70% zoom
            worksheet.freeze_panes(1, 0) # Freeze row 1
            worksheet.hide_gridlines(2) # Hide gridlines

            ## Widths

            writer.sheets[hojas[j]].set_column('A:A', 20)
            writer.sheets[hojas[j]].set_column('B:B', 31, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('C:C', 19, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('D:D', 13)
            writer.sheets[hojas[j]].set_column('E:E', 45)
            writer.sheets[hojas[j]].set_column('F:F', 29, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('G:G', 11)
            writer.sheets[hojas[j]].set_column('H:H', 20, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('I:I', 18)
            writer.sheets[hojas[j]].set_column('J:J', 8, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('L:L', 41, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('M:M', 16)
            writer.sheets[hojas[j]].set_column('N:N', 15)
            writer.sheets[hojas[j]].set_column('O:O', 60, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('P:P', 12)
            writer.sheets[hojas[j]].set_column('Q:Q', 18)
            writer.sheets[hojas[j]].set_column('R:R', 15)
            writer.sheets[hojas[j]].set_column('S:S', 10)
            writer.sheets[hojas[j]].set_column('T:T', 14)
            writer.sheets[hojas[j]].set_column('U:U', 15)
            writer.sheets[hojas[j]].set_column('V:V', 12)
            writer.sheets[hojas[j]].set_column('W:W', 15)
            writer.sheets[hojas[j]].set_column('X:X', 9)
            writer.sheets[hojas[j]].set_column('Y:Y', 9)

    ## Filtros

    writer.sheets[hojas[0]].autofilter(0, 0, resultado_ventas.shape[0], resultado_ventas.shape[1]-1) 
    writer.sheets[hojas[1]].autofilter(0, 0, resultado_egresos.shape[0], resultado_egresos.shape[1]-1) 

    ## Guardar Excel
    writer.save()

else:
    
    print("Solo se procesará los reportes para sellers")

In [40]:
if reporte_semanal_sellers!='yes':

    kams_lista = kams_contactos['Nombre  Completo'].astype(str).drop_duplicates().to_list()

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + r'\\Reportes por KAM'

    for i in range(len(kams_lista)):

        # 1. Definir

        kam = kams_contactos[kams_contactos['Nombre  Completo']==kams_lista[i]]['Nombre  Completo'].drop_duplicates().values[0]
        rep_ventas_kam = resultado_ventas.query("KAM.notnull()")
        rep_ventas_kam = rep_ventas_kam[rep_ventas_kam['KAM'].str.contains(kams_lista[i])].reset_index(drop=True)
        rep_ventas_kam = rep_ventas_kam[rep_ventas_kam['Fecha de Compra']>=fecha_inicio]

        lista = [rep_ventas_kam]
        hojas = ['BASE FACTURA']

        writer = pd.ExcelWriter(os.path.join(final_path, ('Reporte de liquidaciones') + ' - ' + str(kams_lista[i]) + '.xlsx'), engine='xlsxwriter')
            
        ## Exportar Excel
        for j, df in enumerate(lista):
            ## Excel
            df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
            ## Define workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[hojas[j]]
            ## Header format
            header_format = workbook.add_format({'bold': True, 'text_wrap': False, 'valign': 'top', 'fg_color': '#D7E4BC'})
            ## Write headers with the defined format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            ## Format Workbook
            workbook.formats[0].set_font_size(10) # Tamaño 10 letra
            ## Format Worksheet
            worksheet.set_zoom(70) # 70% zoom
            worksheet.freeze_panes(1, 0) # Freeze row 1
            worksheet.hide_gridlines(2) # Hide gridlines

            ## Widths

            writer.sheets[hojas[j]].set_column('A:A', 20)
            writer.sheets[hojas[j]].set_column('B:B', 31, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('C:C', 19, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('D:D', 13)
            writer.sheets[hojas[j]].set_column('E:E', 45)
            writer.sheets[hojas[j]].set_column('F:F', 29, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('G:G', 11)
            writer.sheets[hojas[j]].set_column('H:H', 20, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('I:I', 18)
            writer.sheets[hojas[j]].set_column('J:J', 8, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('L:L', 41, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('M:M', 16)
            writer.sheets[hojas[j]].set_column('N:N', 15)
            writer.sheets[hojas[j]].set_column('O:O', 60, None, {'level': 1, 'hidden': True})
            writer.sheets[hojas[j]].set_column('P:P', 12)
            writer.sheets[hojas[j]].set_column('Q:Q', 18)
            writer.sheets[hojas[j]].set_column('R:R', 15)
            writer.sheets[hojas[j]].set_column('S:S', 10)
            writer.sheets[hojas[j]].set_column('T:T', 14)
            writer.sheets[hojas[j]].set_column('U:U', 15)
            writer.sheets[hojas[j]].set_column('V:V', 12)
            writer.sheets[hojas[j]].set_column('W:W', 15)
            writer.sheets[hojas[j]].set_column('X:X', 9)
            writer.sheets[hojas[j]].set_column('Y:Y', 9)

            ## Filtros

            writer.sheets[hojas[j]].autofilter(0, 0, rep_ventas_kam.shape[0], rep_ventas_kam.shape[1]-1)

        ## Guardar Excel
        writer.save()

else:
    
    print("Solo se procesará los reportes para sellers")

##### **2. Templates de recaudos y liquidaciones semanales**

In [41]:
if reporte_semanal_sellers!='yes':

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','')

    ## Listado de tablas a exportar
    lista = [rec_fcom, total_liq_reg_fcom]
    hojas = ['Recaudos', 'Liquidación']

    ## Nombre del archivo Excel
    writer = pd.ExcelWriter(os.path.join(final_path, ('Templates Recaudos + Liquidación.xlsx')), engine='xlsxwriter')
        
    ## Exportar Excel
    for j, df in enumerate(lista):
        ## Excel
        df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
        ## Define workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets[hojas[j]]
        ## Header format
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
        ## Write headers with the defined format
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        ## Format Workbook
        workbook.formats[0].set_font_size(10) # Tamaño 10 letra
        ## Format Worksheet
        worksheet.set_zoom(70) # 70% zoom
        worksheet.freeze_panes(1, 0) # Freeze row 1
        worksheet.hide_gridlines(2) # Hide gridlines

    ## Widths

    ### 1. Recaudos

    writer.sheets[hojas[0]].set_column('A:A', 23)
    writer.sheets[hojas[0]].set_column('B:B', 20)
    writer.sheets[hojas[0]].set_column('C:C', 26)
    writer.sheets[hojas[0]].set_column('D:D', 10)
    writer.sheets[hojas[0]].set_column('E:E', 9)
    writer.sheets[hojas[0]].set_column('F:F', 14)
    writer.sheets[hojas[0]].set_column('G:G', 13)
    writer.sheets[hojas[0]].set_column('H:H', 12)
    writer.sheets[hojas[0]].set_column('I:I', 14)
    writer.sheets[hojas[0]].set_column('J:J', 16)
    writer.sheets[hojas[0]].set_column('K:K', 22)
    writer.sheets[hojas[0]].set_column('L:L', 16)
    writer.sheets[hojas[0]].set_column('M:M', 12)
    writer.sheets[hojas[0]].set_column('N:N', 50)
    writer.sheets[hojas[0]].set_column('O:O', 13)
    writer.sheets[hojas[0]].set_column('P:P', 20)
    writer.sheets[hojas[0]].set_column('Q:Q', 20)
    writer.sheets[hojas[0]].set_column('R:R', 18)

    ### 2. Liquidación

    writer.sheets[hojas[1]].set_column('A:A', 23)
    writer.sheets[hojas[1]].set_column('B:B', 20)
    writer.sheets[hojas[1]].set_column('C:C', 26)
    writer.sheets[hojas[1]].set_column('D:D', 10)
    writer.sheets[hojas[1]].set_column('E:E', 9)
    writer.sheets[hojas[1]].set_column('F:F', 16)
    writer.sheets[hojas[1]].set_column('G:G', 12)
    writer.sheets[hojas[1]].set_column('H:H', 50)
    writer.sheets[hojas[1]].set_column('I:I', 22)
    writer.sheets[hojas[1]].set_column('J:J', 12)
    writer.sheets[hojas[1]].set_column('K:K', 13)
    writer.sheets[hojas[1]].set_column('L:L', 12)
    writer.sheets[hojas[1]].set_column('M:M', 14)
    writer.sheets[hojas[1]].set_column('N:N', 15)
    writer.sheets[hojas[1]].set_column('O:O', 20)
    writer.sheets[hojas[1]].set_column('P:P', 18)
    writer.sheets[hojas[1]].set_column('Q:Q', 15)
    writer.sheets[hojas[1]].set_column('R:R', 13)
    writer.sheets[hojas[1]].set_column('S:S', 20)
    writer.sheets[hojas[1]].set_column('T:T', 20)
    writer.sheets[hojas[1]].set_column('U:U', 18)

    ## Filtros

    writer.sheets[hojas[0]].autofilter(0, 0, rec_fcom.shape[0], rec_fcom.shape[1]-1) 
    writer.sheets[hojas[1]].autofilter(0, 0, total_liq_reg_fcom.shape[0], total_liq_reg_fcom.shape[1]-1) 

    ## Guardar Excel
    writer.save()

##### **3. Cruce SRX vs Recaudos F.com**

In [42]:
if reporte_semanal_sellers != 'yes':

    ## Ruta
    final_path = ruta_pagos +  r'\\0. Reportes\\2. Reportes CSV\\' + str(pd.to_datetime(fecha_inicio, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','') + '-' + str(pd.to_datetime(fecha_fin, format='%Y%m%d', errors='coerce')).replace('-','').replace(' 00:00:00','')

    ## Listado de tablas a exportar
    lista = [validacion]
    hojas = ['data']

    ## Nombre del archivo Excel
    writer = pd.ExcelWriter(os.path.join(final_path, ('Recaudos SRX - F.com.xlsx')), engine='xlsxwriter')
        
    ## Exportar Excel
    for j, df in enumerate(lista):
        ## Excel
        df.to_excel(writer, sheet_name=hojas[j], index=False)
        ## Define workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets[hojas[j]]
        ## Format Workbook
        workbook.formats[0].set_font_size(10) # Tamaño 10 letra
        ## Format Worksheet
        worksheet.set_zoom(70) # 70% zoom
        worksheet.freeze_panes(1, 0) # Freeze row 1
        worksheet.hide_gridlines(2) # Hide gridlines

    ## Guardar Excel
    writer.save()

##### **4. Reporte consolidado de liquidaciones Falabella.com para team Pagos**

In [47]:
if reporte_semanal_sellers != 'yes':

    final_path = ruta_pagos + r'\\0. Reportes' 
    
     ## CalcFecha de nicio
    fecha_reporte=pd.to_datetime('2022-07-11')

    ## Calcular semana máxima reporte
    max_week = facturas_seller['Semana Recaudación'].max()

    ## Calcular fecha de compra máxima
    max_fecha_compra = rec_seller.query("`Fecha de Compra`.notnull()").loc[:, ['Semana Recaudación', 'Fecha de Compra']].drop_duplicates().groupby(['Semana Recaudación'], dropna=False).agg({'Fecha de Compra': 'max'}).reset_index().query("`Semana Recaudación` == @max_week")['Fecha de Compra'].max()

    ## Filtros
    rep_facturas_seller = facturas_seller
    rep_rec_seller = rec_seller.query("`Semana Recaudación` <= @max_week & `Fecha de Compra` >= @fecha_reporte")
    rep_liq_seller = liq_seller.query("`Semana Recaudación` <= @max_week & `Fecha de Compra` >= @fecha_reporte")
    rep_rec_sin_seller = rec_sin_seller.query("`Semana Recaudación` <= @max_week")
    rep_ajust_seller = ajust_seller.query("`Semana Recaudación` <= @max_week")
    rep_devoluciones_seller = devoluciones_seller.copy() #query("`Fecha de Compra` >= @fecha_coe & `Fecha de Compra` <= @max_fecha_compra")

    ## Listado de tablas a exportar
    lista = [rep_facturas_seller, rep_rec_seller, rep_liq_seller, rep_rec_sin_seller, rep_ajust_seller, rep_devoluciones_seller]
    hojas = ['Facturas', 'Recaudos', 'Liquidaciones', 'Rechazos', 'Ajustes', 'Devoluciones']
    ## Nombre del archivo Excel
    try:
        writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de pagos y devoluciones - Falabella.com' + '.xlsx'), engine='xlsxwriter')
    except:
        writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de pagos y devoluciones - Falabella.com' + str(pd.to_datetime('today').date()) + '.xlsx'), engine='xlsxwriter')
        
    ## Exportar Excel
    for j, df in enumerate(lista):
        ## Excel
        df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
        ## Define workbook and worksheet
        workbook = writer.book
        worksheet = writer.sheets[hojas[j]]
        ## Header format
        header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
        ## Write headers with the defined format
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, header_format)
        ## Format Workbook
        workbook.formats[0].set_font_size(10) # Tamaño 10 letra
        ## Format Worksheet
        worksheet.set_zoom(70) # 70% zoom
        worksheet.freeze_panes(1, 0) # Freeze row 1
        worksheet.hide_gridlines(2) # Hide gridlines

    ## Widths

    ### 1. Facturas

    writer.sheets[hojas[0]].set_column('A:A', 23)
    writer.sheets[hojas[0]].set_column('B:B', 16)
    writer.sheets[hojas[0]].set_column('C:C', 13)
    writer.sheets[hojas[0]].set_column('D:D', 55)
    writer.sheets[hojas[0]].set_column('E:E', 14)
    writer.sheets[hojas[0]].set_column('F:F', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('G:G', 36, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('H:H', 17)
    writer.sheets[hojas[0]].set_column('I:I', 11)
    writer.sheets[hojas[0]].set_column('J:J', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('K:K', 12, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('L:L', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('M:M', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('N:N', 17, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[0]].set_column('O:O', 55)

    ### 2. Recaudos

    writer.sheets[hojas[1]].set_column('A:A', 23)
    writer.sheets[hojas[1]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('E:E', 12)
    writer.sheets[hojas[1]].set_column('F:F', 20)
    writer.sheets[hojas[1]].set_column('G:G', 18)
    writer.sheets[hojas[1]].set_column('H:H', 15)
    writer.sheets[hojas[1]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('M:M', 24)
    writer.sheets[hojas[1]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('Q:Q', 60)
    writer.sheets[hojas[1]].set_column('R:R', 12)
    writer.sheets[hojas[1]].set_column('S:S', 18)
    writer.sheets[hojas[1]].set_column('T:T', 15)
    writer.sheets[hojas[1]].set_column('U:U', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('V:V', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[1]].set_column('W:W', 8)

    ### 3. Liquidaciones

    writer.sheets[hojas[2]].set_column('A:A', 23)
    writer.sheets[hojas[2]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('E:E', 12)
    writer.sheets[hojas[2]].set_column('F:F', 20)
    writer.sheets[hojas[2]].set_column('G:G', 18)
    writer.sheets[hojas[2]].set_column('H:H', 15)
    writer.sheets[hojas[2]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('M:M', 24)
    writer.sheets[hojas[2]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('Q:Q', 60)
    writer.sheets[hojas[2]].set_column('R:R', 12)
    writer.sheets[hojas[2]].set_column('S:S', 18)
    writer.sheets[hojas[2]].set_column('T:T', 15)
    writer.sheets[hojas[2]].set_column('U:U', 15)
    writer.sheets[hojas[2]].set_column('V:V', 20)
    writer.sheets[hojas[2]].set_column('W:W', 16)
    writer.sheets[hojas[2]].set_column('X:X', 8)
    writer.sheets[hojas[2]].set_column('Y:Y', 15)
    writer.sheets[hojas[2]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[2]].set_column('AB:AB', 8)

    ### 4. Rechazos

    writer.sheets[hojas[3]].set_column('A:A', 23)
    writer.sheets[hojas[3]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('E:E', 12)
    writer.sheets[hojas[3]].set_column('F:F', 20)
    writer.sheets[hojas[3]].set_column('G:G', 18)
    writer.sheets[hojas[3]].set_column('H:H', 15)
    writer.sheets[hojas[3]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('M:M', 21)
    writer.sheets[hojas[3]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('Q:Q', 60)
    writer.sheets[hojas[3]].set_column('R:R', 12)
    writer.sheets[hojas[3]].set_column('S:S', 18)
    writer.sheets[hojas[3]].set_column('T:T', 15)
    writer.sheets[hojas[3]].set_column('U:U', 15)
    writer.sheets[hojas[3]].set_column('V:V', 20)
    writer.sheets[hojas[3]].set_column('W:W', 16)
    writer.sheets[hojas[3]].set_column('X:X', 8)
    writer.sheets[hojas[3]].set_column('Y:Y', 18)
    writer.sheets[hojas[3]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[3]].set_column('AB:AB', 8)

    ### 5. Ajustes

    writer.sheets[hojas[4]].set_column('A:A', 23)
    writer.sheets[hojas[4]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('E:E', 12)
    writer.sheets[hojas[4]].set_column('F:F', 20)
    writer.sheets[hojas[4]].set_column('G:G', 18)
    writer.sheets[hojas[4]].set_column('H:H', 15)
    writer.sheets[hojas[4]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('M:M', 24)
    writer.sheets[hojas[4]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('Q:Q', 60)
    writer.sheets[hojas[4]].set_column('R:R', 12)
    writer.sheets[hojas[4]].set_column('S:S', 18)
    writer.sheets[hojas[4]].set_column('T:T', 15)
    writer.sheets[hojas[4]].set_column('U:U', 15)
    writer.sheets[hojas[4]].set_column('V:V', 20)
    writer.sheets[hojas[4]].set_column('W:W', 16)
    writer.sheets[hojas[4]].set_column('X:X', 8)
    writer.sheets[hojas[4]].set_column('Y:Y', 15)
    writer.sheets[hojas[4]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[4]].set_column('AB:AB', 18)

    ### 6. Devoluciones

    writer.sheets[hojas[5]].set_column('A:A', 16, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('B:B', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('C:C', 55, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('D:D', 12)
    writer.sheets[hojas[5]].set_column('E:E', 20)
    writer.sheets[hojas[5]].set_column('F:F', 18)
    writer.sheets[hojas[5]].set_column('G:G', 15)
    writer.sheets[hojas[5]].set_column('H:H', 21)
    writer.sheets[hojas[5]].set_column('I:I', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('J:J', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('K:K', 20)
    writer.sheets[hojas[5]].set_column('L:L', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('M:M', 37, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('N:N', 38, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('O:O', 34, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('P:P', 60)
    writer.sheets[hojas[5]].set_column('Q:Q', 20, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('R:R', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('S:S', 17, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('T:T', 19, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('U:U', 50, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('V:V', 50, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('W:W', 50, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('X:X', 75, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[5]].set_column('Y:Y', 21)
    writer.sheets[hojas[5]].set_column('Z:Z', 10)
    writer.sheets[hojas[5]].set_column('AA:AA', 11)

    ## Filtros

    writer.sheets[hojas[0]].autofilter(0, 0, rep_facturas_seller.shape[0], rep_facturas_seller.shape[1]-1) 
    writer.sheets[hojas[1]].autofilter(0, 0, rep_rec_seller.shape[0], rep_rec_seller.shape[1]-1) 
    writer.sheets[hojas[2]].autofilter(0, 0, rep_liq_seller.shape[0], rep_liq_seller.shape[1]-1) 
    writer.sheets[hojas[3]].autofilter(0, 0, rep_rec_sin_seller.shape[0], rep_rec_sin_seller.shape[1]-1)
    writer.sheets[hojas[4]].autofilter(0, 0, rep_ajust_seller.shape[0], rep_ajust_seller.shape[1]-1)
    writer.sheets[hojas[5]].autofilter(0, 0, rep_devoluciones_seller.shape[0], rep_devoluciones_seller.shape[1]-1)

    ## Guardar Excel
    writer.save()

##### **5. Reporte consolidado de transacciones Falabella.com con Devoluciones**

In [ ]:
## Ruta
final_path = ruta_pagos + r'\\0. Reportes' 

## Listado de tablas a exportar
lista = [rep_transacciones]
hojas = ['data_transacciones']

## Nombre del archivo Excel
try:
    writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de transacciones - Falabella.com' + '.xlsx'), engine='xlsxwriter')
except:
    writer = pd.ExcelWriter(os.path.join(final_path, 'Reporte general de transacciones - Falabella.com' + str(pd.to_datetime('today').date()) + '.xlsx'), engine='xlsxwriter')

## Exportar Excel
for j, df in enumerate(lista):
    ## Excel
    df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
    ## Define workbook and worksheet
    workbook = writer.book
    worksheet = writer.sheets[hojas[j]]
    ## Header format
    header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
    ## Write headers with the defined format
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    ## Format Workbook
    workbook.formats[0].set_font_size(10) # Tamaño 10 letra
    ## Format Worksheet
    worksheet.set_zoom(70) # 70% zoom
    worksheet.freeze_panes(1, 0) # Freeze row 1
    worksheet.hide_gridlines(2) # Hide gridlines

    ## Widths

    ### 1. Data Transacciones

    writer.sheets[hojas[j]].set_column('A:A', 20)
    writer.sheets[hojas[j]].set_column('B:B', 16)
    writer.sheets[hojas[j]].set_column('C:C', 13)
    writer.sheets[hojas[j]].set_column('D:D', 51)
    writer.sheets[hojas[j]].set_column('E:E', 11)
    writer.sheets[hojas[j]].set_column('F:F', 20)
    writer.sheets[hojas[j]].set_column('G:G', 18)
    writer.sheets[hojas[j]].set_column('H:H', 15)
    writer.sheets[hojas[j]].set_column('I:I', 15)
    writer.sheets[hojas[j]].set_column('J:J', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('K:K', 10, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('L:L', 8, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('M:M', 7, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('N:N', 21, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('O:O', 18, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('P:P', 17, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('Q:Q', 14, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('R:R', 13, None, {'level': 1, 'hidden': True})
    writer.sheets[hojas[j]].set_column('S:S', 12)
    writer.sheets[hojas[j]].set_column('T:T', 18)
    writer.sheets[hojas[j]].set_column('U:U', 15)
    writer.sheets[hojas[j]].set_column('V:V', 11)
    writer.sheets[hojas[j]].set_column('W:W', 10)
    writer.sheets[hojas[j]].set_column('X:X', 17)
    writer.sheets[hojas[j]].set_column('Y:Y', 9)
    writer.sheets[hojas[j]].set_column('Z:Z', 14)
    writer.sheets[hojas[j]].set_column('AA:AA', 14)
    writer.sheets[hojas[j]].set_column('AB:AB', 12)
    writer.sheets[hojas[j]].set_column('AC:AC', 12)
    writer.sheets[hojas[j]].set_column('AD:AD', 12)
    writer.sheets[hojas[j]].set_column('AE:AE', 26)

## Filtros

writer.sheets[hojas[0]].autofilter(0, 0, rep_transacciones.shape[0], rep_transacciones.shape[1]-1) 

## Guardar Excel     
writer.save()

##### **6. Reporte semanales de liquidaciones Falabella.com para sellers**

In [ ]:
# 7. Reporte masivo semanal

if reporte_semanal_sellers!='no':

    max_week = facturas_seller.query("`Concepto`=='LIQUIDACIÓN' & `Documento`.notnull()")['Semana Recaudación'].max()
    final_path6 = ruta_pagos +  r'\\0. Reportes\\6. Reportes Semanales - Sellers\\2. Cortes\\' + str(max_week)

    #id_svl = ['5676']
    id_svl = facturas_seller['ID SELLER SVL'].astype(str).drop_duplicates().to_list()
    semanas = resumen_general['Semana Recaudación'].astype(str).drop_duplicates().to_list()

    for i in range(len(id_svl)):

        # 1. Definir

        ## Nombre del seller
        seller = facturas_seller[facturas_seller['ID SELLER SVL']==id_svl[i]]['Seller'].drop_duplicates().values[0]

        ## Calcular semana máxima reporte
        max_week = facturas_seller.query("`Concepto`=='LIQUIDACIÓN' & `Documento`.notnull()")['Semana Recaudación'].max()
        max_week_seller = template_rec[['Semana Recaudación']].sort_values(['Semana Recaudación'], ascending=False).drop_duplicates().max().values[0]

        ## Calcular fecha de compra máxima
        max_fecha_compra = rec_seller \
                                .query("`Fecha de Compra`.notnull()") \
                                .loc[:, ['Semana Recaudación', 'Fecha de Compra']].drop_duplicates() \
                                .groupby(['Semana Recaudación'], dropna=False) \
                                .agg({'Fecha de Compra': 'max'}).reset_index() \
                                .query("`Semana Recaudación` == @max_week_seller")['Fecha de Compra'].max()

        ## Filtros (se quita index sino no funciona con xlsxwriter)
        rep_resumen_seller = resumen_general[(resumen_general['ID SELLER SVL']==id_svl[i]) & (resumen_general['Semana Recaudación'] <= max_week_seller)].reset_index(drop=True).drop(['ID SELLER SVL', 'RUC seller', 'Seller'], axis=1)
        rep_facturas_seller = facturas_seller[facturas_seller['ID SELLER SVL']==id_svl[i]].reset_index(drop=True).drop(['Comentario'], axis=1)
        rep_rec_seller = rec_seller[(rec_seller['ID SELLER SVL']==id_svl[i]) & (rec_seller['Semana Recaudación']<=max_week_seller)].reset_index(drop=True)
        rep_liq_seller = liq_seller[(liq_seller['ID SELLER SVL']==id_svl[i]) & (liq_seller['Semana Recaudación']<=max_week_seller)].reset_index(drop=True)
        rep_rec_sin_seller = rec_sin_seller[(rec_sin_seller['ID SELLER SVL']==id_svl[i]) & (rec_sin_seller['Semana Recaudación']<=max_week_seller)].reset_index(drop=True)
        rep_ajust_seller = ajust_seller[(ajust_seller['ID SELLER SVL']==id_svl[i])]
        rep_devoluciones_seller = devoluciones_seller[(devoluciones_seller['ID SELLER SVL']==id_svl[i])]
        rep_trans_seller = trans_sellers[(trans_sellers['ID SELLER SVL'] == id_svl[i]) & (trans_sellers['Fecha de Compra'] <= max_fecha_compra)].reset_index(drop=True)

        ## Documentos
        try:
          abono = rep_facturas_seller.query("`Semana Recaudación` == @max_week & `Documento` =='FACTURA LIQUIDACIÓN'")['Número de documento'].values[0]
        except:
          abono = ''

        try:
          factura = rep_facturas_seller.query("`Semana Recaudación` == @max_week & `Documento` =='FACTURA COMISIÓN'")['Número de documento'].values[0]
        except:
          factura = ''
        
        try: 
          nc = rep_facturas_seller.query("`Semana Recaudación` == @max_week & `Documento` =='NOTA DE CRÉDITO COMISIÓN'")['Número de documento'].values[0]
        except:
          nc = ''

        # 2. Excel

        ## Listado de tablas a exportar
        lista = [rep_facturas_seller, rep_rec_seller, rep_liq_seller, rep_rec_sin_seller, rep_ajust_seller, rep_devoluciones_seller]
        hojas = ['Facturas', 'Recaudos', 'Liquidaciones', 'Rechazos', 'Ajustes', 'Devoluciones']

        ## Nombre del archivo Excel
        writer = pd.ExcelWriter(os.path.join(final_path6, ('FALABELLA MP - ' + str(id_svl[i]) + ' - ' + str(seller) + '.xlsx')), engine='xlsxwriter')

        ## Crear libro
        workbook = writer.book
        worksheet = workbook.add_worksheet('RESUMEN')

        ## Formatos

        title = workbook.add_format({'bold': True})
        data = workbook.add_format({'bold': True, 'align': 'left'})
        data_amount = workbook.add_format({'align': 'center'})
        line_item = workbook.add_format({'align': 'right'})
        total_item = workbook.add_format({'bold': True, 'align': 'center'})

        amount = workbook.add_format({
            'num_format': '_-[$S/-es-PE] * #,##0.00_-;-[$S/-es-PE] * #,##0.00_-;_-[$S/-es-PE] * "-"??_-;_-@_-',
            'align': 'center'})

        amount_total = workbook.add_format({
            'num_format': '_-[$S/-es-PE] * #,##0.00_-;-[$S/-es-PE] * #,##0.00_-;_-[$S/-es-PE] * "-"??_-;_-@_-',
            'align': 'center', 'bold': True})

        worksheet.set_column('S:S', None, None, {'hidden': True})

        ## Lista de rango de fechas

        for row_num, date in enumerate(semanas):
            worksheet.write(row_num, 18, date)

        worksheet.data_validation('D4', {'validate': 'list', 'source': '=$S$1:$S$90'})

        ## Escribir celdas y formulas

        worksheet.write('C1', 'Estado de Cuenta', title)

        worksheet.write('C3', 'RUC', data)
        worksheet.write_formula('D3', '=Facturas!$C$2', data_amount)

        worksheet.write('C4', 'Semana Recaudación (Periodo)', data)
        worksheet.write('D4', str(max_week), data_amount)

        ## Saldo inicial

        worksheet.write('B6', 'Saldo inicial', title)

        worksheet.write('C6', 'Balance negativo de estados de cuenta anteriores', line_item)
        worksheet.write_formula(
            'D6', '=IF(SUMIFS(Facturas!J:J,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN")-SUMIFS(Facturas!I:I,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN")>=-1,0,SUMIFS(Facturas!J:J,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN")-SUMIFS(Facturas!I:I,Facturas!$A:$A,"<"&RESUMEN!$D$4,Facturas!$E:$E,"LIQUIDACIÓN"))', amount_total)

        ## Órdenes

        worksheet.write('B8', 'Órdenes', title)

        worksheet.write('C8', 'Ingresos por ventas', line_item)
        worksheet.write_formula('D8', '=SUMIFS(Liquidaciones!U:U,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Ingreso")', amount)

        worksheet.write('C9', 'Reembolso de comisión', line_item)
        worksheet.write_formula('D9', '=-SUMIFS(Liquidaciones!$V:$V,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Egreso")', amount)

        worksheet.write('C10', 'Reembolso por Cofinanciamiento Logístico', line_item)
        worksheet.write_formula('D10', '=-SUMIFS(Liquidaciones!$W:$W,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Egreso")', amount)

        worksheet.write('C11', 'Subtotal', total_item)
        worksheet.write_formula('D11', '=D8+D9+D10', amount_total)

        ## Cobros

        worksheet.write('B13', 'Cobros', title)

        worksheet.write('C13', 'Órdenes devueltas', line_item)
        worksheet.write_formula('D13', '=SUMIFS(Liquidaciones!$U:$U,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Egreso")', amount)

        worksheet.write('C14', 'Comisión por venta', line_item)
        worksheet.write_formula('D14', '=-SUMIFS(Facturas!K:K,Facturas!E:E,"COMISIÓN",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C15', 'Cofinanciamiento logístico', line_item)
        worksheet.write_formula('D15', '=-SUMIFS(Liquidaciones!$W:$W,Liquidaciones!$A:$A,RESUMEN!$D$4,Liquidaciones!$AB:$AB,"Ingreso")', amount)

        worksheet.write('C16', 'Regularizaciones (Otros Cobros)', line_item)
        worksheet.write_formula('D16', '=SUMIFS(Facturas!O:O,Facturas!E:E,"LIQUIDACIÓN",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C17', 'Subtotal', total_item)
        worksheet.write_formula('D17', '=D13+D14+D15+D16', amount_total)

        ## Reembolsos

        worksheet.write('B19', 'Reembolsos', title)

        worksheet.write('C19', 'Devoluciones (Rechazos y Siniestros)', line_item)
        worksheet.write_formula('D19', '=SUMIFS(Facturas!M:M,Facturas!E:E,"DEVOLUCIONES",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C20', 'Ajustes (Errores de cobros y precios)', line_item)
        worksheet.write_formula('D20', '=SUMIFS(Facturas!N:N,Facturas!E:E,"AJUSTES",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C21', 'Subtotal', total_item)
        worksheet.write_formula('D21', '=D19+D20', amount_total)

        ## Saldo final

        worksheet.write('B23', 'Saldo final', title)

        worksheet.write('C23', 'Balance final', total_item)
        worksheet.write_formula('D23', '=D6+D11+D17+D21', amount_total)

        ## Pagos

        worksheet.write('B25', 'Pagos', title)

        worksheet.write('C25', 'Importe abonado (Liquidaciones)', line_item)
        worksheet.write_formula('D25', '=SUMIFS(Facturas!I:I,Facturas!E:E,"LIQUIDACIÓN",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C26', 'Importe abonado (Reembolsos)', line_item)
        worksheet.write_formula('D26', '=SUMIFS(Facturas!M:M,Facturas!E:E,"DEVOLUCIONES",Facturas!A:A,RESUMEN!$D$4)', amount)

        worksheet.write('C27', 'Pago total', total_item)
        worksheet.write_formula('D27', '=D25+D26', amount_total)

        ## Format Worksheet
        worksheet.set_zoom(70)
        worksheet.hide_gridlines(2)
        worksheet.set_column('A:A', 1)
        worksheet.set_column('B:B', 11)
        worksheet.set_column('C:C', 47)
        worksheet.set_column('D:D', 19)

        ## --------- FIN CODIGO SAMUEL

        ## Exportar Excel
        for j, df in enumerate(lista):
            ## Excel
            df.to_excel(writer, sheet_name=hojas[j], startrow=1, index=False, header=False)
            ## Define workbook and worksheet
            workbook = writer.book
            worksheet = writer.sheets[hojas[j]]
            ## Header format
            header_format = workbook.add_format({'bold': True, 'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC'})
            ## Write headers with the defined format
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)
            ## Format Workbook
            workbook.formats[0].set_font_size(10) # Tamaño 10 letra
            ## Format Worksheet
            worksheet.set_zoom(70) # 70% zoom
            worksheet.freeze_panes(1, 0) # Freeze row 1
            worksheet.hide_gridlines(2) # Hide gridlines
            
        ## Widths

        ### 1. Facturas

        writer.sheets[hojas[0]].set_column('A:A', 23)
        writer.sheets[hojas[0]].set_column('B:B', 16)
        writer.sheets[hojas[0]].set_column('C:C', 13)
        writer.sheets[hojas[0]].set_column('D:D', 55)
        writer.sheets[hojas[0]].set_column('E:E', 14)
        writer.sheets[hojas[0]].set_column('F:F', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('G:G', 36, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('H:H', 17)
        writer.sheets[hojas[0]].set_column('I:I', 11)
        writer.sheets[hojas[0]].set_column('J:J', 14, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('K:K', 12, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('L:L', 14, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('M:M', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('N:N', 10, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('O:O', 17, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[0]].set_column('P:P', 10)

        ### 2. Recaudos

        writer.sheets[hojas[1]].set_column('A:A', 23)
        writer.sheets[hojas[1]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('E:E', 12)
        writer.sheets[hojas[1]].set_column('F:F', 20)
        writer.sheets[hojas[1]].set_column('G:G', 18)
        writer.sheets[hojas[1]].set_column('H:H', 15)
        writer.sheets[hojas[1]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('M:M', 24)
        writer.sheets[hojas[1]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('Q:Q', 60)
        writer.sheets[hojas[1]].set_column('R:R', 12)
        writer.sheets[hojas[1]].set_column('S:S', 18)
        writer.sheets[hojas[1]].set_column('T:T', 15)
        writer.sheets[hojas[1]].set_column('U:U', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('V:V', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[1]].set_column('W:W', 8)

        ### 3. Liquidaciones

        writer.sheets[hojas[2]].set_column('A:A', 23)
        writer.sheets[hojas[2]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('E:E', 12)
        writer.sheets[hojas[2]].set_column('F:F', 20)
        writer.sheets[hojas[2]].set_column('G:G', 18)
        writer.sheets[hojas[2]].set_column('H:H', 15)
        writer.sheets[hojas[2]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('M:M', 24)
        writer.sheets[hojas[2]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('Q:Q', 60)
        writer.sheets[hojas[2]].set_column('R:R', 12)
        writer.sheets[hojas[2]].set_column('S:S', 18)
        writer.sheets[hojas[2]].set_column('T:T', 15)
        writer.sheets[hojas[2]].set_column('U:U', 15)
        writer.sheets[hojas[2]].set_column('V:V', 20)
        writer.sheets[hojas[2]].set_column('W:W', 16)
        writer.sheets[hojas[2]].set_column('X:X', 8)
        writer.sheets[hojas[2]].set_column('Y:Y', 15)
        writer.sheets[hojas[2]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[2]].set_column('AB:AB', 8)

        ### 4. Rechazos

        writer.sheets[hojas[3]].set_column('A:A', 23)
        writer.sheets[hojas[3]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('E:E', 12)
        writer.sheets[hojas[3]].set_column('F:F', 20)
        writer.sheets[hojas[3]].set_column('G:G', 18)
        writer.sheets[hojas[3]].set_column('H:H', 15)
        writer.sheets[hojas[3]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('M:M', 21)
        writer.sheets[hojas[3]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('Q:Q', 60)
        writer.sheets[hojas[3]].set_column('R:R', 12)
        writer.sheets[hojas[3]].set_column('S:S', 18)
        writer.sheets[hojas[3]].set_column('T:T', 15)
        writer.sheets[hojas[3]].set_column('U:U', 15)
        writer.sheets[hojas[3]].set_column('V:V', 20)
        writer.sheets[hojas[3]].set_column('W:W', 16)
        writer.sheets[hojas[3]].set_column('X:X', 8)
        writer.sheets[hojas[3]].set_column('Y:Y', 18)
        writer.sheets[hojas[3]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[3]].set_column('AB:AB', 8)

        ### 5. Ajustes

        writer.sheets[hojas[4]].set_column('A:A', 23)
        writer.sheets[hojas[4]].set_column('B:B', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('C:C', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('D:D', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('E:E', 12)
        writer.sheets[hojas[4]].set_column('F:F', 20)
        writer.sheets[hojas[4]].set_column('G:G', 18)
        writer.sheets[hojas[4]].set_column('H:H', 15)
        writer.sheets[hojas[4]].set_column('I:I', 23, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('J:J', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('K:K', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('L:L', 25, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('M:M', 24)
        writer.sheets[hojas[4]].set_column('N:N', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('O:O', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('P:P', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('Q:Q', 60)
        writer.sheets[hojas[4]].set_column('R:R', 12)
        writer.sheets[hojas[4]].set_column('S:S', 18)
        writer.sheets[hojas[4]].set_column('T:T', 15)
        writer.sheets[hojas[4]].set_column('U:U', 15)
        writer.sheets[hojas[4]].set_column('V:V', 20)
        writer.sheets[hojas[4]].set_column('W:W', 16)
        writer.sheets[hojas[4]].set_column('X:X', 8)
        writer.sheets[hojas[4]].set_column('Y:Y', 15)
        writer.sheets[hojas[4]].set_column('Z:Z', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('AA:AA', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[4]].set_column('AB:AB', 18)

        ### 6. Devoluciones

        writer.sheets[hojas[5]].set_column('A:A', 16, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('B:B', 13, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('C:C', 55, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('D:D', 12)
        writer.sheets[hojas[5]].set_column('E:E', 20)
        writer.sheets[hojas[5]].set_column('F:F', 18)
        writer.sheets[hojas[5]].set_column('G:G', 15)
        writer.sheets[hojas[5]].set_column('H:H', 21)
        writer.sheets[hojas[5]].set_column('I:I', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('J:J', 18, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('K:K', 20)
        writer.sheets[hojas[5]].set_column('L:L', 21, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('M:M', 37, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('N:N', 38, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('O:O', 34, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('P:P', 60)
        writer.sheets[hojas[5]].set_column('Q:Q', 20, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('R:R', 14, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('S:S', 17, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('T:T', 19, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('U:U', 50, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('V:V', 50, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('W:W', 50, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('X:X', 75, None, {'level': 1, 'hidden': True})
        writer.sheets[hojas[5]].set_column('Y:Y', 21)
        writer.sheets[hojas[5]].set_column('Z:Z', 10)
        writer.sheets[hojas[5]].set_column('AA:AA', 11)

        ## Filtros

        ### 1. Facturas

        ##### Activar filtro
        writer.sheets[hojas[0]].autofilter(0, 0, rep_facturas_seller.shape[0], rep_facturas_seller.shape[1]-1) 
        writer.sheets[hojas[0]].filter_column_list('A', max_week)
        writer.sheets[hojas[0]].filter_column_list('E', ['LIQUIDACIÓN'])   

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_facturas_seller.iterrows():
            region = row_data['Semana Recaudación']
            volume = row_data['Concepto']
            if not (region == max_week and volume == 'LIQUIDACIÓN'):
                writer.sheets[hojas[0]].set_row(row + 1, options={'hidden': True})

        ### 2. Recaudos

        ##### Activar filtro
        writer.sheets[hojas[1]].autofilter(0, 0, rep_rec_seller.shape[0], rep_rec_seller.shape[1]-1) 
        writer.sheets[hojas[1]].filter_column_list('A', max_week)
        
        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_rec_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[1]].set_row(row + 1, options={'hidden': True})

        ### 3. Liquidaciones

        ##### Activar filtro
        writer.sheets[hojas[2]].autofilter(0, 0, rep_liq_seller.shape[0], rep_liq_seller.shape[1]-1) 
        writer.sheets[hojas[2]].filter_column_list('A', max_week)

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_liq_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[2]].set_row(row + 1, options={'hidden': True})

        ### 4. Rechazos

        ##### Activar filtro
        writer.sheets[hojas[3]].autofilter(0, 0, rep_rec_sin_seller.shape[0], rep_rec_sin_seller.shape[1]-1) 
        writer.sheets[hojas[3]].filter_column_list('A', max_week)

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_rec_sin_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[3]].set_row(row + 1, options={'hidden': True})

        ### 5. Ajustes

        ##### Activar filtro
        writer.sheets[hojas[4]].autofilter(0, 0, rep_ajust_seller.shape[0], rep_ajust_seller.shape[1]-1)
        writer.sheets[hojas[4]].filter_column_list('A', max_week)

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_ajust_seller.iterrows():
            region = row_data['Semana Recaudación']
            if not (region == max_week):
                writer.sheets[hojas[4]].set_row(row + 1, options={'hidden': True})

        ### 6. Devoluciones

        writer.sheets[hojas[5]].autofilter(0, 0, rep_devoluciones_seller.shape[0], rep_devoluciones_seller.shape[1]-1)
        writer.sheets[hojas[5]].filter_column_list('H', ['ENTREGADO'])

        ##### Esconder rows que no cumplen con el filtro
        for row, row_data in rep_devoluciones_seller.iterrows():
            volume = row_data['Estado Egreso']
            if not (volume == 'ENTREGADO'):
                writer.sheets[hojas[5]].set_row(row + 1, options={'hidden': True})    

        ## Guardar Excel     
        writer.save()

        # 3. PDF

        ## 1. Resumen seller

        x = liq_seller[((liq_seller['ID SELLER SVL'] == id_svl[i])) & (liq_seller['Semana Recaudación']==max_week)]

        ## 2. Resumen de liquidaciones

        ### 1. Ingresos
        x_ingreso = x.query("Tipo=='Ingreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Liquidación']] \
                    .rename(columns={'Liquidación': 'Pago'}) \
                    .assign(**{'Pago': lambda x: x['Pago'].apply(lambda x: float("{:.2f}".format(x)))})

        ### 2. Egresos
        x_egreso = x.query("Tipo=='Egreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Liquidación']] \
                    .rename(columns={'Liquidación': 'Descuento'}) \
                    .assign(**{'Descuento': lambda x: x['Descuento'].apply(lambda x: float("{:.2f}".format(x)))})

        ### 3. Resumen
        x_resumen = pd.merge(x_ingreso, x_egreso, how='outer', on=['Folio Relacionado', 'Producto (SKU)']) \
                      .groupby(['Folio Relacionado', 'Producto (SKU)'], dropna=False) \
                      .agg({'Pago': 'sum', 'Descuento': 'sum'}).reset_index() \
                      .assign(**{'Liquidación': lambda x: x['Pago'] + x['Descuento']}) \
                      .sort_values(['Folio Relacionado']) \
                      .assign(**{'Liquidación': lambda x: x['Liquidación'].apply(lambda x: float("{:.2f}".format(x)))})

        ## 3. Resumen de facturas

        x_factura = x.query("Tipo=='Ingreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Importe Comisión', 'Cof. Logístico']] \
                    .assign(**{
                      'Importe Comisión': lambda x: x['Importe Comisión'].apply(lambda x: float("{:.2f}".format(x))),
                      'Cof. Logístico': lambda x: x['Cof. Logístico'].apply(lambda x: float("{:.2f}".format(x)))
                    })

        ## 4. Resumen de notas de crédito

        x_nc = x.query("Tipo=='Egreso'") \
                    .loc[:, ['Folio Relacionado', 'Producto (SKU)', 'Importe Comisión', 'Cof. Logístico']] \
                    .assign(**{
                      'Importe Comisión': lambda x: x['Importe Comisión'].apply(lambda x: float("{:.2f}".format(x))),
                      'Cof. Logístico': lambda x: x['Cof. Logístico'].apply(lambda x: float("{:.2f}".format(x)))
                    })

        ## 5. Crear PDF

        pdf = FPDF()

        ### 1. Liquidaciones

        #### Agregar hoja
        pdf.add_page()

        #### Title
        pdf.set_font('arial', 'B', 11)
        pdf.cell(50)
        pdf.cell(75, 10, 'Resumen de Liquidaciones - ' + str(abono), 0, 2, 'C')
        pdf.cell(50, 5, ' ', 0, 2, 'C')
        pdf.cell(-45)

        #### Headers
        pdf.set_font('arial', 'B', 9)
        columnNameList = list(x_resumen.columns)
        for header in columnNameList[:-1]:
          pdf.cell(35, 6, header, 1, 0, 'C')
        pdf.cell(35, 6, columnNameList[-1], 1, 2, 'C')

        #### Values
        pdf.cell(-140)
        pdf.set_font('arial', '', 8)
        for row in range(0, len(x_resumen)):
          for col_num, col_name in enumerate(columnNameList):
            if col_num != len(columnNameList)-1:
              pdf.cell(35, 4, str(x_resumen[col_name].iloc[row]), 1, 0, 'C')
            else:
              pdf.cell(35, 4, str(x_resumen[col_name].iloc[row]), 1, 2, 'R')  
              pdf.cell(-140)

        ### 2. Factura

        #### Agregar hoja
        pdf.add_page()

        #### Title
        pdf.set_font('arial', 'B', 11)
        pdf.cell(50)
        pdf.cell(75, 10, 'Resumen de importe de la factura - ' + str(factura), 0, 2, 'C')
        pdf.cell(50, 5, ' ', 0, 2, 'C')
        pdf.cell(-45)

        #### Headers
        pdf.set_font('arial', 'B', 9)
        col_factura = list(x_factura.columns)
        for header in col_factura[:-1]:
          pdf.cell(35, 6, header, 1, 0, 'C')
        pdf.cell(35, 6, col_factura[-1], 1, 2, 'C')

        #### Values
        pdf.cell(-105)
        pdf.set_font('arial', '', 8)
        for row in range(0, len(x_factura)):
          for col_num, col_name in enumerate(col_factura):
            if col_num != len(col_factura)-1:
              pdf.cell(35, 4, str(x_factura[col_name].iloc[row]), 1, 0, 'C')
            else:
              pdf.cell(35, 4, str(x_factura[col_name].iloc[row]), 1, 2, 'R')  
              pdf.cell(-105)

        ### 3. Nota de crédito

        #### Agregar hoja
        pdf.add_page()

        #### Title
        pdf.set_font('arial', 'B', 11)
        pdf.cell(50)
        pdf.cell(75, 10, 'Resumen de importe de la nota de crédito - ' + str(nc), 0, 2, 'C')
        pdf.cell(50, 5, ' ', 0, 2, 'C')
        pdf.cell(-45)

        #### Headers
        pdf.set_font('arial', 'B', 9)
        col_nc = list(x_nc.columns)
        for header in col_nc[:-1]:
          pdf.cell(35, 6, header, 1, 0, 'C')
        pdf.cell(35, 6, col_nc[-1], 1, 2, 'C')

        #### Values
        pdf.cell(-105)
        pdf.set_font('arial', '', 8)
        for row in range(0, len(x_nc)):
          for col_num, col_name in enumerate(col_nc):
            if col_num != len(col_nc)-1:
              pdf.cell(35, 4, str(x_nc[col_name].iloc[row]), 1, 0, 'C')
            else:
              pdf.cell(35, 4, str(x_nc[col_name].iloc[row]), 1, 2, 'R')
              pdf.cell(-105)

        #### Output
        pdf.output(os.path.join(final_path6, ('FALABELLA MP - ' + str(id_svl[i]) + ' - ' + str(seller) + '.pdf')), 'F') 

else:
    
    print("Para tener este reporte es necesario habilitar la opción de reportes sellers")